## Loading Best Acc list from Txt

In [1]:
#best_acc_eval_results_by_layer = {}
#best_f1_eval_results_by_layer = {}
import ast

with open('L12_Frozen_acc.txt') as file:
    data = file.read()    
file.close()
    
best_acc_eval_results_by_layer = ast.literal_eval(data)


with open('L12_Frozen_f1.txt') as file:
    data = file.read()    
file.close()


best_f1_eval_results_by_layer = ast.literal_eval(data)


In [36]:
print('Best Acc List:\n', best_acc_eval_results_by_layer)
print()
print('Best F1 List:\n', best_f1_eval_results_by_layer)

Best Acc List:
 {(12, 0): {'acc': 0.8580357142857142, 'f1': 0.7823453075568864}, (12, 0.25): {'acc': 0.8625, 'f1': 0.7960469155284686}, (12, 0.75): {'acc': 0.8544642857142857, 'f1': 0.7955473836394455}, (11, 0): {'acc': 0.8625, 'f1': 0.7983648942079483}, (10, 0): {'acc': 0.8526785714285714, 'f1': 0.7791258718917419}, (9, 0): {'acc': 0.8482142857142857, 'f1': 0.7775968480864099}, (8, 0): {'acc': 0.8383928571428572, 'f1': 0.7648445547046677}, (7, 0): {'acc': 0.8258928571428571, 'f1': 0.7403418274383831}, (6, 0): {'acc': 0.8089285714285714, 'f1': 0.7116461710857843}, (5, 0): {'acc': 0.7857142857142857, 'f1': 0.6710740356981434}, (4, 0): {'acc': 0.7848214285714286, 'f1': 0.6839390708793694}, (3, 0): {'acc': 0.7848214285714286, 'f1': 0.6624438214965042}, (2, 0): {'acc': 0.7758928571428572, 'f1': 0.6818424550257919}, (1, 0): {'acc': 0.7633928571428571, 'f1': 0.6241070849483733}, (0, 0): {'acc': 0.65, 'f1': 0.26262626262626265}, (12, 0.9): {'acc': 0.8375, 'f1': 0.7580973192536766}, (12, 0.5):

### Save Dictionary 

In [37]:
with open('L12_Frozen_acc.txt', 'w') as file:
    file.write(str(best_acc_eval_results_by_layer))
file.close()
    
with open('L12_Frozen_f1.txt', 'w') as file:
    file.write(str(best_f1_eval_results_by_layer))
file.close()

### Device Check

In [3]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.current_device()
torch.cuda.device(0)


## Loading Dataset, Model

In [4]:
# coding=utf-8
import argparse
import logging
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import random

import numpy as np
import pandas as pd
import torch
from transformers import (BertConfig, BertForTokenClassification,
                                  BertTokenizer)
from torch.utils.data import DataLoader

from datasets import load_datasets_and_vocabs
from model import (Aspect_Text_GAT_ours,
                    Pure_Bert, Aspect_Bert_GAT, Aspect_Text_GAT_only)
from trainer import train

logger = logging.getLogger(__name__)

def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed_all(args.seed)

def parse_args(args):
    parser = argparse.ArgumentParser()

    # Required parameters
    parser.add_argument('--dataset_name', type=str, default='rest',
                        choices=['rest', 'laptop', 'twitter'],
                        help='Choose absa dataset.')
    parser.add_argument('--output_dir', type=str, default='/data1/SHENWZH/ABSA_online/data/output-gcn',
                        help='Directory to store intermedia data, such as vocab, embeddings, tags_vocab.')
    parser.add_argument('--num_classes', type=int, default=3,
                        help='Number of classes of ABSA.')


    parser.add_argument('--cuda_id', type=str, default='3',
                        help='Choose which GPUs to run')
    parser.add_argument('--seed', type=int, default=2019,
                        help='random seed for initialization')

    # Model parameters
    parser.add_argument('--glove_dir', type=str, default='/data1/SHENWZH/wordvec',
                        help='Directory storing glove embeddings')
    parser.add_argument('--bert_model_dir', type=str, default='/data1/SHENWZH/models/bert_base',
                        help='Path to pre-trained Bert model.')
    parser.add_argument('--pure_bert', action='store_true',
                        help='Cat text and aspect, [cls] to predict.')
    
    parser.add_argument('--pure_bert_layer_agg', action='store_true',
                        help='Pure bert layer aggregation enable/not')

    parser.add_argument('--pure_bert_layer_agg_list', type=str, default="12",
                        help='Pure Bert layer number to aggregate')

    parser.add_argument('--pure_bert_linear_layer_count', type=int, default=2,
                        help='Pure Bert final linear layer count')
    
    parser.add_argument('--gat_bert', action='store_true',
                        help='Cat text and aspect, [cls] to predict.')

    parser.add_argument('--highway', action='store_true',
                        help='Use highway embed.')

    parser.add_argument('--num_layers', type=int, default=2,
                        help='Number of layers of bilstm or highway or elmo.')
    
    parser.add_argument('--hidden_layer_no', type=int, default=12,
                        help='NO of hidder layer of bert state output to FC.')
 
    parser.add_argument('--frozen_percent', type=float, default = 0.1,
                       help='Percentage of the frozen layers.')


    parser.add_argument('--add_non_connect',  type= bool, default=True,
                        help='Add a sepcial "non-connect" relation for aspect with no direct connection.')
    parser.add_argument('--multi_hop',  type= bool, default=True,
                        help='Multi hop non connection.')
    parser.add_argument('--max_hop', type = int, default=4,
                        help='max number of hops')


    parser.add_argument('--num_heads', type=int, default=6,
                        help='Number of heads for gat.')
    
    parser.add_argument('--dropout', type=float, default=0,
                        help='Dropout rate for embedding.')


    parser.add_argument('--num_gcn_layers', type=int, default=1,
                        help='Number of GCN layers.')
    parser.add_argument('--gcn_mem_dim', type=int, default=300,
                        help='Dimension of the W in GCN.')
    parser.add_argument('--gcn_dropout', type=float, default=0.2,
                        help='Dropout rate for GCN.')
    # GAT
    parser.add_argument('--gat', action='store_true',
                        help='GAT')
    parser.add_argument('--gat_our', action='store_true',
                        help='GAT_our')
    parser.add_argument('--gat_attention_type', type = str, choices=['linear','dotprod','gcn'], default='dotprod',
                        help='The attention used for gat')

    parser.add_argument('--embedding_type', type=str,default='glove', choices=['glove','bert'])
    parser.add_argument('--embedding_dim', type=int, default=300,
                        help='Dimension of glove embeddings')
    parser.add_argument('--dep_relation_embed_dim', type=int, default=300,
                        help='Dimension for dependency relation embeddings.')

    parser.add_argument('--hidden_size', type=int, default=300,
                        help='Hidden size of bilstm, in early stage.')
    parser.add_argument('--final_hidden_size', type=int, default=300,
                        help='Hidden size of bilstm, in early stage.')
    parser.add_argument('--num_mlps', type=int, default=2,
                        help='Number of mlps in the last of model.')

    # Training parameters
    parser.add_argument("--per_gpu_train_batch_size", default=16, type=int,
                        help="Batch size per GPU/CPU for training.")
    parser.add_argument("--per_gpu_eval_batch_size", default=32, type=int,
                        help="Batch size per GPU/CPU for evaluation.")
    parser.add_argument('--gradient_accumulation_steps', type=int, default=2,
                        help="Number of updates steps to accumulate before performing a backward/update pass.")
    parser.add_argument("--learning_rate", default=1e-3, type=float,
                        help="The initial learning rate for Adam.")
    
    parser.add_argument("--weight_decay", default=0.0, type=float,
                        help="Weight deay if we apply some.")
    parser.add_argument("--adam_epsilon", default=1e-8, type=float,
                        help="Epsilon for Adam optimizer.")

    parser.add_argument("--max_grad_norm", default=1.0, type=float,
                        help="Max gradient norm.")
    parser.add_argument("--num_train_epochs", default=30.0, type=float,
                        help="Total number of training epochs to perform.")
    parser.add_argument("--max_steps", default=-1, type=int,
                        help="If > 0: set total number of training steps(that update the weights) to perform. Override num_train_epochs.")
    parser.add_argument('--logging_steps', type=int, default=50,
                        help="Log every X updates steps.")
    
    return parser.parse_args(args)


def check_args(args):
    '''
    eliminate confilct situations
    
    '''
    logger.info(vars(args))

/home/ubuntu/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.6) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [6]:
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                        datefmt='%m/%d/%Y %H:%M:%S',
                        level=logging.INFO)
    
# Parse args

args_str = "--embedding_type bert --output_dir data/output-gcn --dropout 0.3 --hidden_size 200 --learning_rate 5e-5 --bert_model_dir ./test/saved_model --pure_bert --hidden_layer_no 12 --frozen_percent 0"
#args = parse_args(['--gat_our', '--highway', '--num_heads', '7', '--dropout', '0.8', '--output_dir',
#                   'output/r-gat', '--glove_dir', 'glove', '--cuda_id', '0'])
args = parse_args(args_str.split(' '))
check_args(args)

# Setup CUDA, GPU training
os.environ["CUDA_VISIBLE_DEVICES"] = args.cuda_id
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
args.device = device
logger.info('Device is %s', args.device)

# Set seed
set_seed(args)

# Bert, load pretrained model and tokenizer, check if neccesary to put bert here
if args.embedding_type == 'bert':
    tokenizer = BertTokenizer.from_pretrained(args.bert_model_dir)
    args.tokenizer = tokenizer

# Load datasets and vocabs
train_dataset, test_dataset, word_vocab, dep_tag_vocab, pos_tag_vocab= load_datasets_and_vocabs(args)




07/23/2021 19:45:20 - INFO - __main__ -   {'dataset_name': 'rest', 'output_dir': 'data/output-gcn', 'num_classes': 3, 'cuda_id': '3', 'seed': 2019, 'glove_dir': '/data1/SHENWZH/wordvec', 'bert_model_dir': './test/saved_model', 'pure_bert': True, 'pure_bert_layer_agg': False, 'pure_bert_layer_agg_list': '12', 'pure_bert_linear_layer_count': 2, 'gat_bert': False, 'highway': False, 'num_layers': 2, 'hidden_layer_no': 12, 'frozen_percent': 0.0, 'add_non_connect': True, 'multi_hop': True, 'max_hop': 4, 'num_heads': 6, 'dropout': 0.3, 'num_gcn_layers': 1, 'gcn_mem_dim': 300, 'gcn_dropout': 0.2, 'gat': False, 'gat_our': False, 'gat_attention_type': 'dotprod', 'embedding_type': 'bert', 'embedding_dim': 300, 'dep_relation_embed_dim': 300, 'hidden_size': 200, 'final_hidden_size': 300, 'num_mlps': 2, 'per_gpu_train_batch_size': 16, 'per_gpu_eval_batch_size': 32, 'gradient_accumulation_steps': 2, 'learning_rate': 5e-05, 'weight_decay': 0.0, 'adam_epsilon': 1e-08, 'max_grad_norm': 1.0, 'num_train_e

# Pure Bert - Single Layer 

## Output of Bert, output[12] All Frozen

In [11]:
args.hidden_layer_no = 12
args.frozen_percent = 1

# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, arg.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, arg.frozen_percent)] = best_f1_eval_result

07/23/2021 02:14:52 - INFO - model -   Hidden Layer 12
07/23/2021 02:14:52 - INFO - trainer -   ***** Running training *****
07/23/2021 02:14:52 - INFO - trainer -     Num examples = 3602
07/23/2021 02:14:52 - INFO - trainer -     Num Epochs = 30
07/23/2021 02:14:52 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 02:14:52 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 02:14:52 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 02:14:55 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 02:14:55 - INFO - trainer -     Num examples = 1120
07/23/2021 02:14:55 - INFO - trainer -     Batch size = 32
07/23/2021 02:14:56 - INFO - trainer -   ***** Eval results *****
07/23/2021 02:14:56 - INFO - trainer -     eval loss: 0.9064929451261248
07/23/2021 02:14:56 - INFO - trainer -     acc = 0.6491071428571429
07/23/2021 02:14:56 - INFO - trainer -     f1 = 0.2624075076700957
07/23/202

07/23/2021 02:15:48 - INFO - trainer -     acc = 0.66875
07/23/2021 02:15:48 - INFO - trainer -     f1 = 0.35051252443039344
07/23/2021 02:15:51 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 02:15:51 - INFO - trainer -     Num examples = 1120
07/23/2021 02:15:51 - INFO - trainer -     Batch size = 32
07/23/2021 02:15:52 - INFO - trainer -   ***** Eval results *****
07/23/2021 02:15:52 - INFO - trainer -     eval loss: 0.7815773725509644
07/23/2021 02:15:52 - INFO - trainer -     acc = 0.6660714285714285
07/23/2021 02:15:52 - INFO - trainer -     f1 = 0.34528654773689427
07/23/2021 02:15:54 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 02:15:54 - INFO - trainer -     Num examples = 1120
07/23/2021 02:15:54 - INFO - trainer -     Batch size = 32
07/23/2021 02:15:55 - INFO - trainer -   ***** Eval results *****
07/23/2021 02:15:55 - INFO - trainer -     eval loss: 0.7860433169773646
07/23/2021 02:15:55 - INFO - trainer -     acc = 0.6767857142857143
07/23

07/23/2021 02:16:48 - INFO - trainer -     eval loss: 0.7194710178034646
07/23/2021 02:16:48 - INFO - trainer -     acc = 0.7044642857142858
07/23/2021 02:16:48 - INFO - trainer -     f1 = 0.46891240565291886
Epoch:  50%|█████     | 15/30 [01:57<01:56,  7.75s/it]07/23/2021 02:16:50 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 02:16:50 - INFO - trainer -     Num examples = 1120
07/23/2021 02:16:50 - INFO - trainer -     Batch size = 32
07/23/2021 02:16:51 - INFO - trainer -   ***** Eval results *****
07/23/2021 02:16:51 - INFO - trainer -     eval loss: 0.7247870666640145
07/23/2021 02:16:51 - INFO - trainer -     acc = 0.7008928571428571
07/23/2021 02:16:51 - INFO - trainer -     f1 = 0.45426988613989755
07/23/2021 02:16:53 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 02:16:53 - INFO - trainer -     Num examples = 1120
07/23/2021 02:16:53 - INFO - trainer -     Batch size = 32
07/23/2021 02:16:55 - INFO - trainer -   ***** Eval results *****
07/23/20

07/23/2021 02:17:47 - INFO - trainer -     Batch size = 32
07/23/2021 02:17:49 - INFO - trainer -   ***** Eval results *****
07/23/2021 02:17:49 - INFO - trainer -     eval loss: 0.6892816594668797
07/23/2021 02:17:49 - INFO - trainer -     acc = 0.7160714285714286
07/23/2021 02:17:49 - INFO - trainer -     f1 = 0.4912712749682577
07/23/2021 02:17:51 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 02:17:51 - INFO - trainer -     Num examples = 1120
07/23/2021 02:17:51 - INFO - trainer -     Batch size = 32
07/23/2021 02:17:52 - INFO - trainer -   ***** Eval results *****
07/23/2021 02:17:52 - INFO - trainer -     eval loss: 0.6935524501970836
07/23/2021 02:17:52 - INFO - trainer -     acc = 0.7089285714285715
07/23/2021 02:17:52 - INFO - trainer -     f1 = 0.4695821032834326
Epoch:  77%|███████▋  | 23/30 [03:02<00:56,  8.09s/it]07/23/2021 02:17:55 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 02:17:55 - INFO - trainer -     Num examples = 1120
07/23/2021

07/23/2021 02:18:47 - INFO - trainer -     Num examples = 1120
07/23/2021 02:18:47 - INFO - trainer -     Batch size = 32
07/23/2021 02:18:48 - INFO - trainer -   ***** Eval results *****
07/23/2021 02:18:48 - INFO - trainer -     eval loss: 0.6775206297636032
07/23/2021 02:18:48 - INFO - trainer -     acc = 0.7267857142857143
07/23/2021 02:18:48 - INFO - trainer -     f1 = 0.5151736707334873
Epoch: 100%|██████████| 30/30 [03:57<00:00,  7.92s/it]
07/23/2021 02:18:50 - INFO - __main__ -     acc = 0.7366071428571429
07/23/2021 02:18:50 - INFO - __main__ -     f1 = 0.5572514607994633


### Output of Bert, output[12] Frozen 0

In [4]:
args.hidden_layer_no = 12
args.frozen_percent = 0
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_f1_eval_result

07/23/2021 03:00:39 - INFO - model -   Hidden Layer 12 frozen percentage 0
07/23/2021 03:00:41 - INFO - trainer -   ***** Running training *****
07/23/2021 03:00:41 - INFO - trainer -     Num examples = 3602
07/23/2021 03:00:41 - INFO - trainer -     Num Epochs = 30
07/23/2021 03:00:41 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 03:00:41 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 03:00:41 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 03:00:49 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:00:49 - INFO - trainer -     Num examples = 1120
07/23/2021 03:00:49 - INFO - trainer -     Batch size = 32
07/23/2021 03:00:50 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:00:50 - INFO - trainer -     eval loss: 0.6072911794696535
07/23/2021 03:00:50 - INFO - trainer -     acc = 0.7607142857142857
07/23/2021 03:00:50 - INFO - trainer -     f1 = 0.545665

07/23/2021 03:03:00 - INFO - trainer -     acc = 0.8580357142857142
07/23/2021 03:03:00 - INFO - trainer -     f1 = 0.7823453075568864
07/23/2021 03:03:08 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:03:08 - INFO - trainer -     Num examples = 1120
07/23/2021 03:03:08 - INFO - trainer -     Batch size = 32
07/23/2021 03:03:09 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:03:09 - INFO - trainer -     eval loss: 0.8051507401280105
07/23/2021 03:03:09 - INFO - trainer -     acc = 0.8410714285714286
07/23/2021 03:03:09 - INFO - trainer -     f1 = 0.7556317379138645
07/23/2021 03:03:17 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:03:17 - INFO - trainer -     Num examples = 1120
07/23/2021 03:03:17 - INFO - trainer -     Batch size = 32
07/23/2021 03:03:18 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:03:18 - INFO - trainer -     eval loss: 0.8362530197183202
07/23/2021 03:03:18 - INFO - trainer -     acc = 0.8392857142857

07/23/2021 03:05:27 - INFO - trainer -     eval loss: 1.0138292685467085
07/23/2021 03:05:27 - INFO - trainer -     acc = 0.8517857142857143
07/23/2021 03:05:27 - INFO - trainer -     f1 = 0.7839190006750095
Epoch:  50%|█████     | 15/30 [04:52<04:48, 19.22s/it]07/23/2021 03:05:34 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:05:34 - INFO - trainer -     Num examples = 1120
07/23/2021 03:05:34 - INFO - trainer -     Batch size = 32
07/23/2021 03:05:36 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:05:36 - INFO - trainer -     eval loss: 1.0893574889233735
07/23/2021 03:05:36 - INFO - trainer -     acc = 0.8410714285714286
07/23/2021 03:05:36 - INFO - trainer -     f1 = 0.7543556908053791
07/23/2021 03:05:43 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:05:43 - INFO - trainer -     Num examples = 1120
07/23/2021 03:05:43 - INFO - trainer -     Batch size = 32
07/23/2021 03:05:44 - INFO - trainer -   ***** Eval results *****
07/23/2021

07/23/2021 03:07:50 - INFO - trainer -     Batch size = 32
07/23/2021 03:07:51 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:07:51 - INFO - trainer -     eval loss: 1.1743714654606134
07/23/2021 03:07:51 - INFO - trainer -     acc = 0.8321428571428572
07/23/2021 03:07:51 - INFO - trainer -     f1 = 0.761935087584944
07/23/2021 03:07:58 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:07:58 - INFO - trainer -     Num examples = 1120
07/23/2021 03:07:58 - INFO - trainer -     Batch size = 32
07/23/2021 03:07:59 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:07:59 - INFO - trainer -     eval loss: 1.2112254739244235
07/23/2021 03:07:59 - INFO - trainer -     acc = 0.8366071428571429
07/23/2021 03:07:59 - INFO - trainer -     f1 = 0.7583271659048078
Epoch:  77%|███████▋  | 23/30 [07:25<02:12, 18.95s/it]07/23/2021 03:08:06 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:08:06 - INFO - trainer -     Num examples = 1120
07/23/2021 

07/23/2021 03:10:14 - INFO - trainer -     Num examples = 1120
07/23/2021 03:10:14 - INFO - trainer -     Batch size = 32
07/23/2021 03:10:15 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:10:15 - INFO - trainer -     eval loss: 1.15045785567823
07/23/2021 03:10:15 - INFO - trainer -     acc = 0.8392857142857143
07/23/2021 03:10:15 - INFO - trainer -     f1 = 0.7704319640260014
Epoch: 100%|██████████| 30/30 [09:39<00:00, 19.31s/it]
07/23/2021 03:10:20 - INFO - __main__ -     acc = 0.8580357142857142
07/23/2021 03:10:20 - INFO - __main__ -     f1 = 0.7823453075568864


NameError: name 'best_acc_eval_results_by_layer' is not defined

In [7]:
best_acc_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_acc_eval_result
best_f1_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_f1_eval_result

In [8]:
best_acc_eval_results_by_layer

{(12, 0): {'acc': 0.8580357142857142, 'f1': 0.7823453075568864}}

In [9]:
best_f1_eval_results_by_layer

{(12, 0): {'acc': 0.8580357142857142, 'f1': 0.794648953589351}}

### Output of Bert, output[12] Frozen 0.1

In [35]:
args.hidden_layer_no = 12
args.frozen_percent = 0.1
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_f1_eval_result

07/23/2021 22:00:01 - INFO - model -   Hidden Layer 12 frozen percentage 0.1
07/23/2021 22:00:01 - INFO - trainer -   ***** Running training *****
07/23/2021 22:00:01 - INFO - trainer -     Num examples = 3602
07/23/2021 22:00:01 - INFO - trainer -     Num Epochs = 30
07/23/2021 22:00:01 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 22:00:01 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 22:00:01 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 22:00:08 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 22:00:08 - INFO - trainer -     Num examples = 1120
07/23/2021 22:00:08 - INFO - trainer -     Batch size = 32
07/23/2021 22:00:09 - INFO - trainer -   ***** Eval results *****
07/23/2021 22:00:09 - INFO - trainer -     eval loss: 0.5813058908496584
07/23/2021 22:00:09 - INFO - trainer -     acc = 0.7598214285714285
07/23/2021 22:00:09 - INFO - trainer -     f1 = 0.5596

07/23/2021 22:02:10 - INFO - trainer -     acc = 0.8517857142857143
07/23/2021 22:02:10 - INFO - trainer -     f1 = 0.7812867581131694
07/23/2021 22:02:17 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 22:02:17 - INFO - trainer -     Num examples = 1120
07/23/2021 22:02:17 - INFO - trainer -     Batch size = 32
07/23/2021 22:02:18 - INFO - trainer -   ***** Eval results *****
07/23/2021 22:02:18 - INFO - trainer -     eval loss: 0.7829165961194251
07/23/2021 22:02:18 - INFO - trainer -     acc = 0.8473214285714286
07/23/2021 22:02:18 - INFO - trainer -     f1 = 0.7725238265783242
07/23/2021 22:02:25 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 22:02:25 - INFO - trainer -     Num examples = 1120
07/23/2021 22:02:25 - INFO - trainer -     Batch size = 32
07/23/2021 22:02:27 - INFO - trainer -   ***** Eval results *****
07/23/2021 22:02:27 - INFO - trainer -     eval loss: 0.7755085055716335
07/23/2021 22:02:27 - INFO - trainer -     acc = 0.8428571428571

07/23/2021 22:04:26 - INFO - trainer -     eval loss: 1.1507861780734467
07/23/2021 22:04:26 - INFO - trainer -     acc = 0.8348214285714286
07/23/2021 22:04:26 - INFO - trainer -     f1 = 0.7434570827113177
Epoch:  50%|█████     | 15/30 [04:31<04:26, 17.79s/it]07/23/2021 22:04:33 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 22:04:33 - INFO - trainer -     Num examples = 1120
07/23/2021 22:04:33 - INFO - trainer -     Batch size = 32
07/23/2021 22:04:34 - INFO - trainer -   ***** Eval results *****
07/23/2021 22:04:34 - INFO - trainer -     eval loss: 1.1823919384108323
07/23/2021 22:04:34 - INFO - trainer -     acc = 0.8294642857142858
07/23/2021 22:04:34 - INFO - trainer -     f1 = 0.7394606395191511
07/23/2021 22:04:41 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 22:04:41 - INFO - trainer -     Num examples = 1120
07/23/2021 22:04:41 - INFO - trainer -     Batch size = 32
07/23/2021 22:04:42 - INFO - trainer -   ***** Eval results *****
07/23/2021

07/23/2021 22:06:40 - INFO - trainer -     Batch size = 32
07/23/2021 22:06:41 - INFO - trainer -   ***** Eval results *****
07/23/2021 22:06:41 - INFO - trainer -     eval loss: 1.1901561822521867
07/23/2021 22:06:41 - INFO - trainer -     acc = 0.8339285714285715
07/23/2021 22:06:41 - INFO - trainer -     f1 = 0.740573213195832
07/23/2021 22:06:48 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 22:06:48 - INFO - trainer -     Num examples = 1120
07/23/2021 22:06:48 - INFO - trainer -     Batch size = 32
07/23/2021 22:06:49 - INFO - trainer -   ***** Eval results *****
07/23/2021 22:06:49 - INFO - trainer -     eval loss: 1.3690346521637236
07/23/2021 22:06:49 - INFO - trainer -     acc = 0.8321428571428572
07/23/2021 22:06:49 - INFO - trainer -     f1 = 0.7345421742718733
Epoch:  77%|███████▋  | 23/30 [06:54<02:04, 17.76s/it]07/23/2021 22:06:56 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 22:06:56 - INFO - trainer -     Num examples = 1120
07/23/2021 

07/23/2021 22:08:54 - INFO - trainer -     Num examples = 1120
07/23/2021 22:08:54 - INFO - trainer -     Batch size = 32
07/23/2021 22:08:56 - INFO - trainer -   ***** Eval results *****
07/23/2021 22:08:56 - INFO - trainer -     eval loss: 1.4187898783281396
07/23/2021 22:08:56 - INFO - trainer -     acc = 0.8348214285714286
07/23/2021 22:08:56 - INFO - trainer -     f1 = 0.7276603962023368
Epoch: 100%|██████████| 30/30 [08:59<00:00, 17.99s/it]
07/23/2021 22:09:01 - INFO - __main__ -     acc = 0.8553571428571428
07/23/2021 22:09:01 - INFO - __main__ -     f1 = 0.7861172628573901


### Output of Bert, output[12] Frozen 0.25

In [10]:
args.hidden_layer_no = 12
args.frozen_percent = 0.25
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_f1_eval_result

07/23/2021 03:15:48 - INFO - model -   Hidden Layer 12 frozen percentage 0.25
07/23/2021 03:15:49 - INFO - trainer -   ***** Running training *****
07/23/2021 03:15:49 - INFO - trainer -     Num examples = 3602
07/23/2021 03:15:49 - INFO - trainer -     Num Epochs = 30
07/23/2021 03:15:49 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 03:15:49 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 03:15:49 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 03:15:54 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:15:54 - INFO - trainer -     Num examples = 1120
07/23/2021 03:15:54 - INFO - trainer -     Batch size = 32
07/23/2021 03:15:56 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:15:56 - INFO - trainer -     eval loss: 0.5889721155166626
07/23/2021 03:15:56 - INFO - trainer -     acc = 0.7821428571428571
07/23/2021 03:15:56 - INFO - trainer -     f1 = 0.620

07/23/2021 03:17:43 - INFO - trainer -     acc = 0.8419642857142857
07/23/2021 03:17:43 - INFO - trainer -     f1 = 0.7656895263010371
07/23/2021 03:17:49 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:17:49 - INFO - trainer -     Num examples = 1120
07/23/2021 03:17:49 - INFO - trainer -     Batch size = 32
07/23/2021 03:17:50 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:17:50 - INFO - trainer -     eval loss: 0.7680219803138503
07/23/2021 03:17:50 - INFO - trainer -     acc = 0.85
07/23/2021 03:17:50 - INFO - trainer -     f1 = 0.7725526024299278
07/23/2021 03:17:56 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:17:56 - INFO - trainer -     Num examples = 1120
07/23/2021 03:17:56 - INFO - trainer -     Batch size = 32
07/23/2021 03:17:57 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:17:57 - INFO - trainer -     eval loss: 0.691530282090285
07/23/2021 03:17:57 - INFO - trainer -     acc = 0.8616071428571429
07/23/2021 

07/23/2021 03:19:43 - INFO - trainer -     acc = 0.8553571428571428
07/23/2021 03:19:43 - INFO - trainer -     f1 = 0.7847646527953435
Epoch:  50%|█████     | 15/30 [03:59<03:57, 15.82s/it]07/23/2021 03:19:49 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:19:49 - INFO - trainer -     Num examples = 1120
07/23/2021 03:19:49 - INFO - trainer -     Batch size = 32
07/23/2021 03:19:50 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:19:50 - INFO - trainer -     eval loss: 1.158260177055906
07/23/2021 03:19:50 - INFO - trainer -     acc = 0.84375
07/23/2021 03:19:50 - INFO - trainer -     f1 = 0.7532528634132382
07/23/2021 03:19:56 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:19:56 - INFO - trainer -     Num examples = 1120
07/23/2021 03:19:56 - INFO - trainer -     Batch size = 32
07/23/2021 03:19:57 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:19:57 - INFO - trainer -     eval loss: 1.1301005523161232
07/23/2021 03:19:57 - 

07/23/2021 03:21:43 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:21:43 - INFO - trainer -     eval loss: 1.083670751673136
07/23/2021 03:21:43 - INFO - trainer -     acc = 0.8455357142857143
07/23/2021 03:21:43 - INFO - trainer -     f1 = 0.7648040728444375
07/23/2021 03:21:49 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:21:49 - INFO - trainer -     Num examples = 1120
07/23/2021 03:21:49 - INFO - trainer -     Batch size = 32
07/23/2021 03:21:50 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:21:50 - INFO - trainer -     eval loss: 1.1377573055543637
07/23/2021 03:21:50 - INFO - trainer -     acc = 0.8357142857142857
07/23/2021 03:21:50 - INFO - trainer -     f1 = 0.7480734054844126
Epoch:  77%|███████▋  | 23/30 [06:07<01:50, 15.83s/it]07/23/2021 03:21:56 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:21:56 - INFO - trainer -     Num examples = 1120
07/23/2021 03:21:56 - INFO - trainer -     Batch size = 32
07/23/2021 

07/23/2021 03:23:42 - INFO - trainer -     Batch size = 32
07/23/2021 03:23:44 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:23:44 - INFO - trainer -     eval loss: 1.2049756014204052
07/23/2021 03:23:44 - INFO - trainer -     acc = 0.83125
07/23/2021 03:23:44 - INFO - trainer -     f1 = 0.7377088804997779
Epoch: 100%|██████████| 30/30 [07:59<00:00, 15.99s/it]
07/23/2021 03:23:48 - INFO - __main__ -     acc = 0.8625
07/23/2021 03:23:48 - INFO - __main__ -     f1 = 0.7960469155284686


### Output of Bert, output[12] Frozen 0.5

In [32]:
args.hidden_layer_no = 12
args.frozen_percent = 0.5
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_f1_eval_result

07/23/2021 21:50:49 - INFO - model -   Hidden Layer 12 frozen percentage 0.5
07/23/2021 21:50:49 - INFO - trainer -   ***** Running training *****
07/23/2021 21:50:49 - INFO - trainer -     Num examples = 3602
07/23/2021 21:50:49 - INFO - trainer -     Num Epochs = 30
07/23/2021 21:50:49 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 21:50:49 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 21:50:49 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 21:50:54 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:50:54 - INFO - trainer -     Num examples = 1120
07/23/2021 21:50:54 - INFO - trainer -     Batch size = 32
07/23/2021 21:50:55 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:50:55 - INFO - trainer -     eval loss: 0.6034779544387545
07/23/2021 21:50:55 - INFO - trainer -     acc = 0.7723214285714286
07/23/2021 21:50:55 - INFO - trainer -     f1 = 0.5679

07/23/2021 21:52:26 - INFO - trainer -     acc = 0.8553571428571428
07/23/2021 21:52:26 - INFO - trainer -     f1 = 0.7807095457637937
07/23/2021 21:52:31 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:52:31 - INFO - trainer -     Num examples = 1120
07/23/2021 21:52:31 - INFO - trainer -     Batch size = 32
07/23/2021 21:52:32 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:52:32 - INFO - trainer -     eval loss: 0.7380651230790786
07/23/2021 21:52:32 - INFO - trainer -     acc = 0.8446428571428571
07/23/2021 21:52:32 - INFO - trainer -     f1 = 0.7743843214761094
07/23/2021 21:52:37 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:52:37 - INFO - trainer -     Num examples = 1120
07/23/2021 21:52:37 - INFO - trainer -     Batch size = 32
07/23/2021 21:52:38 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:52:38 - INFO - trainer -     eval loss: 0.7917135020185794
07/23/2021 21:52:38 - INFO - trainer -     acc = 0.8455357142857

07/23/2021 21:54:08 - INFO - trainer -     eval loss: 1.1416109993567392
07/23/2021 21:54:08 - INFO - trainer -     acc = 0.8366071428571429
07/23/2021 21:54:08 - INFO - trainer -     f1 = 0.7432919212288289
Epoch:  50%|█████     | 15/30 [03:22<03:19, 13.28s/it]07/23/2021 21:54:12 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:54:12 - INFO - trainer -     Num examples = 1120
07/23/2021 21:54:12 - INFO - trainer -     Batch size = 32
07/23/2021 21:54:13 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:54:13 - INFO - trainer -     eval loss: 1.1145888219358833
07/23/2021 21:54:13 - INFO - trainer -     acc = 0.8357142857142857
07/23/2021 21:54:13 - INFO - trainer -     f1 = 0.7502830854067538
07/23/2021 21:54:18 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:54:18 - INFO - trainer -     Num examples = 1120
07/23/2021 21:54:18 - INFO - trainer -     Batch size = 32
07/23/2021 21:54:19 - INFO - trainer -   ***** Eval results *****
07/23/2021

07/23/2021 21:55:47 - INFO - trainer -     Batch size = 32
07/23/2021 21:55:48 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:55:48 - INFO - trainer -     eval loss: 1.284543668571028
07/23/2021 21:55:48 - INFO - trainer -     acc = 0.8357142857142857
07/23/2021 21:55:48 - INFO - trainer -     f1 = 0.7522419266091473
07/23/2021 21:55:53 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:55:53 - INFO - trainer -     Num examples = 1120
07/23/2021 21:55:53 - INFO - trainer -     Batch size = 32
07/23/2021 21:55:54 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:55:54 - INFO - trainer -     eval loss: 1.2800051386733164
07/23/2021 21:55:54 - INFO - trainer -     acc = 0.8508928571428571
07/23/2021 21:55:54 - INFO - trainer -     f1 = 0.7678786693006749
Epoch:  77%|███████▋  | 23/30 [05:09<01:32, 13.21s/it]07/23/2021 21:55:59 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:55:59 - INFO - trainer -     Num examples = 1120
07/23/2021 

07/23/2021 21:57:28 - INFO - trainer -     Num examples = 1120
07/23/2021 21:57:28 - INFO - trainer -     Batch size = 32
07/23/2021 21:57:29 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:57:29 - INFO - trainer -     eval loss: 1.1606569862943226
07/23/2021 21:57:29 - INFO - trainer -     acc = 0.8446428571428571
07/23/2021 21:57:29 - INFO - trainer -     f1 = 0.7698761589074562
Epoch: 100%|██████████| 30/30 [06:43<00:00, 13.46s/it]
07/23/2021 21:57:33 - INFO - __main__ -     acc = 0.8553571428571428
07/23/2021 21:57:33 - INFO - __main__ -     f1 = 0.7807095457637937


### Output of Bert, output[12] Frozen 0.75

In [11]:
args.hidden_layer_no = 12
args.frozen_percent = 0.75
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_f1_eval_result

07/23/2021 03:25:59 - INFO - model -   Hidden Layer 12 frozen percentage 0.75
07/23/2021 03:25:59 - INFO - trainer -   ***** Running training *****
07/23/2021 03:25:59 - INFO - trainer -     Num examples = 3602
07/23/2021 03:25:59 - INFO - trainer -     Num Epochs = 30
07/23/2021 03:25:59 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 03:25:59 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 03:25:59 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 03:26:02 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:26:02 - INFO - trainer -     Num examples = 1120
07/23/2021 03:26:02 - INFO - trainer -     Batch size = 32
07/23/2021 03:26:03 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:26:03 - INFO - trainer -     eval loss: 0.7379045597144536
07/23/2021 03:26:03 - INFO - trainer -     acc = 0.7133928571428572
07/23/2021 03:26:03 - INFO - trainer -     f1 = 0.440

07/23/2021 03:27:13 - INFO - trainer -     acc = 0.84375
07/23/2021 03:27:13 - INFO - trainer -     f1 = 0.7608707862767581
07/23/2021 03:27:17 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:27:17 - INFO - trainer -     Num examples = 1120
07/23/2021 03:27:17 - INFO - trainer -     Batch size = 32
07/23/2021 03:27:18 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:27:18 - INFO - trainer -     eval loss: 0.5863856578245759
07/23/2021 03:27:18 - INFO - trainer -     acc = 0.8464285714285714
07/23/2021 03:27:18 - INFO - trainer -     f1 = 0.7727342910724446
07/23/2021 03:27:21 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:27:21 - INFO - trainer -     Num examples = 1120
07/23/2021 03:27:21 - INFO - trainer -     Batch size = 32
07/23/2021 03:27:23 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:27:23 - INFO - trainer -     eval loss: 0.5644378356635571
07/23/2021 03:27:23 - INFO - trainer -     acc = 0.8455357142857143
07/23/2

07/23/2021 03:28:32 - INFO - trainer -     eval loss: 0.868643713336704
07/23/2021 03:28:32 - INFO - trainer -     acc = 0.8401785714285714
07/23/2021 03:28:32 - INFO - trainer -     f1 = 0.7577496152989128
Epoch:  50%|█████     | 15/30 [02:36<02:35, 10.35s/it]07/23/2021 03:28:36 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:28:36 - INFO - trainer -     Num examples = 1120
07/23/2021 03:28:36 - INFO - trainer -     Batch size = 32
07/23/2021 03:28:37 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:28:37 - INFO - trainer -     eval loss: 0.8478353490055139
07/23/2021 03:28:37 - INFO - trainer -     acc = 0.8455357142857143
07/23/2021 03:28:37 - INFO - trainer -     f1 = 0.7668853511362906
07/23/2021 03:28:40 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:28:40 - INFO - trainer -     Num examples = 1120
07/23/2021 03:28:40 - INFO - trainer -     Batch size = 32
07/23/2021 03:28:42 - INFO - trainer -   ***** Eval results *****
07/23/2021 

07/23/2021 03:29:50 - INFO - trainer -     Batch size = 32
07/23/2021 03:29:51 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:29:51 - INFO - trainer -     eval loss: 0.9041991234291344
07/23/2021 03:29:51 - INFO - trainer -     acc = 0.8473214285714286
07/23/2021 03:29:51 - INFO - trainer -     f1 = 0.7794765554218372
07/23/2021 03:29:54 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:29:54 - INFO - trainer -     Num examples = 1120
07/23/2021 03:29:54 - INFO - trainer -     Batch size = 32
07/23/2021 03:29:56 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:29:56 - INFO - trainer -     eval loss: 1.0004015458747744
07/23/2021 03:29:56 - INFO - trainer -     acc = 0.8383928571428572
07/23/2021 03:29:56 - INFO - trainer -     f1 = 0.7596352277118966
Epoch:  77%|███████▋  | 23/30 [04:00<01:12, 10.32s/it]07/23/2021 03:29:59 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:29:59 - INFO - trainer -     Num examples = 1120
07/23/2021

07/23/2021 03:31:08 - INFO - trainer -     Num examples = 1120
07/23/2021 03:31:08 - INFO - trainer -     Batch size = 32
07/23/2021 03:31:09 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:31:09 - INFO - trainer -     eval loss: 1.2338148049411497
07/23/2021 03:31:09 - INFO - trainer -     acc = 0.8357142857142857
07/23/2021 03:31:09 - INFO - trainer -     f1 = 0.7490991675969849
Epoch: 100%|██████████| 30/30 [05:13<00:00, 10.44s/it]
07/23/2021 03:31:12 - INFO - __main__ -     acc = 0.8544642857142857
07/23/2021 03:31:12 - INFO - __main__ -     f1 = 0.7955473836394455


In [12]:
best_acc_eval_results_by_layer

{(12, 0): {'acc': 0.8580357142857142, 'f1': 0.7823453075568864},
 (12, 0.25): {'acc': 0.8625, 'f1': 0.7960469155284686},
 (12, 0.75): {'acc': 0.8544642857142857, 'f1': 0.7955473836394455}}

In [13]:
best_f1_eval_results_by_layer

{(12, 0): {'acc': 0.8580357142857142, 'f1': 0.794648953589351},
 (12, 0.25): {'acc': 0.8616071428571429, 'f1': 0.798711316849118},
 (12, 0.75): {'acc': 0.8544642857142857, 'f1': 0.7955473836394455}}

### Output of Bert, output[12] Frozen 0.9

In [31]:
args.hidden_layer_no = 12
args.frozen_percent = 0.9
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_f1_eval_result

07/23/2021 21:44:29 - INFO - model -   Hidden Layer 12 frozen percentage 0.9
07/23/2021 21:44:29 - INFO - trainer -   ***** Running training *****
07/23/2021 21:44:29 - INFO - trainer -     Num examples = 3602
07/23/2021 21:44:29 - INFO - trainer -     Num Epochs = 30
07/23/2021 21:44:29 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 21:44:29 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 21:44:29 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 21:44:32 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:44:32 - INFO - trainer -     Num examples = 1120
07/23/2021 21:44:32 - INFO - trainer -     Batch size = 32
07/23/2021 21:44:33 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:44:33 - INFO - trainer -     eval loss: 0.8925355298178537
07/23/2021 21:44:33 - INFO - trainer -     acc = 0.6517857142857143
07/23/2021 21:44:33 - INFO - trainer -     f1 = 0.2697

07/23/2021 21:45:33 - INFO - trainer -     acc = 0.8196428571428571
07/23/2021 21:45:33 - INFO - trainer -     f1 = 0.7252133019087599
07/23/2021 21:45:36 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:45:36 - INFO - trainer -     Num examples = 1120
07/23/2021 21:45:36 - INFO - trainer -     Batch size = 32
07/23/2021 21:45:37 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:45:37 - INFO - trainer -     eval loss: 0.49076588430574963
07/23/2021 21:45:37 - INFO - trainer -     acc = 0.8214285714285714
07/23/2021 21:45:37 - INFO - trainer -     f1 = 0.7320889692680764
07/23/2021 21:45:40 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:45:40 - INFO - trainer -     Num examples = 1120
07/23/2021 21:45:40 - INFO - trainer -     Batch size = 32
07/23/2021 21:45:41 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:45:41 - INFO - trainer -     eval loss: 0.4912434143679483
07/23/2021 21:45:41 - INFO - trainer -     acc = 0.823214285714

07/23/2021 21:46:41 - INFO - trainer -     eval loss: 0.5721568103347506
07/23/2021 21:46:41 - INFO - trainer -     acc = 0.8294642857142858
07/23/2021 21:46:41 - INFO - trainer -     f1 = 0.7482478097373416
Epoch:  50%|█████     | 15/30 [02:15<02:14,  8.95s/it]07/23/2021 21:46:44 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:46:44 - INFO - trainer -     Num examples = 1120
07/23/2021 21:46:44 - INFO - trainer -     Batch size = 32
07/23/2021 21:46:45 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:46:45 - INFO - trainer -     eval loss: 0.5646225491272552
07/23/2021 21:46:45 - INFO - trainer -     acc = 0.8285714285714286
07/23/2021 21:46:45 - INFO - trainer -     f1 = 0.7499838947163641
07/23/2021 21:46:48 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:46:48 - INFO - trainer -     Num examples = 1120
07/23/2021 21:46:48 - INFO - trainer -     Batch size = 32
07/23/2021 21:46:49 - INFO - trainer -   ***** Eval results *****
07/23/2021

07/23/2021 21:47:49 - INFO - trainer -     Batch size = 32
07/23/2021 21:47:50 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:47:50 - INFO - trainer -     eval loss: 0.8022272431424686
07/23/2021 21:47:50 - INFO - trainer -     acc = 0.8151785714285714
07/23/2021 21:47:50 - INFO - trainer -     f1 = 0.7240806097482345
07/23/2021 21:47:53 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:47:53 - INFO - trainer -     Num examples = 1120
07/23/2021 21:47:53 - INFO - trainer -     Batch size = 32
07/23/2021 21:47:54 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:47:54 - INFO - trainer -     eval loss: 0.7403491188905069
07/23/2021 21:47:54 - INFO - trainer -     acc = 0.8285714285714286
07/23/2021 21:47:54 - INFO - trainer -     f1 = 0.7414295842844297
Epoch:  77%|███████▋  | 23/30 [03:28<01:02,  8.97s/it]07/23/2021 21:47:57 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:47:57 - INFO - trainer -     Num examples = 1120
07/23/2021

07/23/2021 21:48:57 - INFO - trainer -     Num examples = 1120
07/23/2021 21:48:57 - INFO - trainer -     Batch size = 32
07/23/2021 21:48:59 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:48:59 - INFO - trainer -     eval loss: 0.9016576081514358
07/23/2021 21:48:59 - INFO - trainer -     acc = 0.81875
07/23/2021 21:48:59 - INFO - trainer -     f1 = 0.731313695379031
Epoch: 100%|██████████| 30/30 [04:32<00:00,  9.07s/it]
07/23/2021 21:49:01 - INFO - __main__ -     acc = 0.8375
07/23/2021 21:49:01 - INFO - __main__ -     f1 = 0.7580973192536766


### Output of Bert, output[11] All Frozen

In [7]:
args.hidden_layer_no = 11
args.frozen_percent = 1
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[args.hidden_layer_no] = best_acc_eval_result
    best_f1_eval_results_by_layer[args.hidden_layer_no] = best_f1_eval_result
    
    
        

07/23/2021 02:04:54 - INFO - model -   Hidden Layer 11
07/23/2021 02:04:55 - INFO - trainer -   ***** Running training *****
07/23/2021 02:04:55 - INFO - trainer -     Num examples = 3602
07/23/2021 02:04:55 - INFO - trainer -     Num Epochs = 30
07/23/2021 02:04:55 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 02:04:55 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 02:04:55 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 02:05:14 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 02:05:14 - INFO - trainer -     Num examples = 1120
07/23/2021 02:05:14 - INFO - trainer -     Batch size = 32
07/23/2021 02:05:16 - INFO - trainer -   ***** Eval results *****
07/23/2021 02:05:16 - INFO - trainer -     eval loss: 0.8898086428642273
07/23/2021 02:05:16 - INFO - trainer -     acc = 0.65
07/23/2021 02:05:16 - INFO - trainer -     f1 = 0.26262626262626265
07/23/2021 02:05:18 - 

07/23/2021 02:06:08 - INFO - trainer -     f1 = 0.3486217611429545
07/23/2021 02:06:10 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 02:06:10 - INFO - trainer -     Num examples = 1120
07/23/2021 02:06:10 - INFO - trainer -     Batch size = 32
07/23/2021 02:06:11 - INFO - trainer -   ***** Eval results *****
07/23/2021 02:06:11 - INFO - trainer -     eval loss: 0.7764740066868918
07/23/2021 02:06:11 - INFO - trainer -     acc = 0.6705357142857142
07/23/2021 02:06:11 - INFO - trainer -     f1 = 0.36833616112009854
07/23/2021 02:06:13 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 02:06:13 - INFO - trainer -     Num examples = 1120
07/23/2021 02:06:13 - INFO - trainer -     Batch size = 32
07/23/2021 02:06:15 - INFO - trainer -   ***** Eval results *****
07/23/2021 02:06:15 - INFO - trainer -     eval loss: 0.774716945205416
07/23/2021 02:06:15 - INFO - trainer -     acc = 0.6723214285714286
07/23/2021 02:06:15 - INFO - trainer -     f1 = 0.36864012338037

07/23/2021 02:07:07 - INFO - trainer -     acc = 0.6991071428571428
07/23/2021 02:07:07 - INFO - trainer -     f1 = 0.4478531441800257
Epoch:  50%|█████     | 15/30 [02:14<01:57,  7.84s/it]07/23/2021 02:07:09 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 02:07:09 - INFO - trainer -     Num examples = 1120
07/23/2021 02:07:09 - INFO - trainer -     Batch size = 32
07/23/2021 02:07:10 - INFO - trainer -   ***** Eval results *****
07/23/2021 02:07:10 - INFO - trainer -     eval loss: 0.7287754723003932
07/23/2021 02:07:10 - INFO - trainer -     acc = 0.6955357142857143
07/23/2021 02:07:10 - INFO - trainer -     f1 = 0.43555824301414664
07/23/2021 02:07:13 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 02:07:13 - INFO - trainer -     Num examples = 1120
07/23/2021 02:07:13 - INFO - trainer -     Batch size = 32
07/23/2021 02:07:14 - INFO - trainer -   ***** Eval results *****
07/23/2021 02:07:14 - INFO - trainer -     eval loss: 0.7314496972731181
07/23/202

07/23/2021 02:08:06 - INFO - trainer -   ***** Eval results *****
07/23/2021 02:08:06 - INFO - trainer -     eval loss: 0.6949426472187042
07/23/2021 02:08:06 - INFO - trainer -     acc = 0.7214285714285714
07/23/2021 02:08:06 - INFO - trainer -     f1 = 0.5206787255820795
07/23/2021 02:08:08 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 02:08:08 - INFO - trainer -     Num examples = 1120
07/23/2021 02:08:08 - INFO - trainer -     Batch size = 32
07/23/2021 02:08:10 - INFO - trainer -   ***** Eval results *****
07/23/2021 02:08:10 - INFO - trainer -     eval loss: 0.7023780235222408
07/23/2021 02:08:10 - INFO - trainer -     acc = 0.7053571428571429
07/23/2021 02:08:10 - INFO - trainer -     f1 = 0.46748953947588423
Epoch:  77%|███████▋  | 23/30 [03:17<00:54,  7.77s/it]07/23/2021 02:08:12 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 02:08:12 - INFO - trainer -     Num examples = 1120
07/23/2021 02:08:12 - INFO - trainer -     Batch size = 32
07/23/202

07/23/2021 02:09:05 - INFO - trainer -     Batch size = 32
07/23/2021 02:09:06 - INFO - trainer -   ***** Eval results *****
07/23/2021 02:09:06 - INFO - trainer -     eval loss: 0.6873542359897069
07/23/2021 02:09:06 - INFO - trainer -     acc = 0.7223214285714286
07/23/2021 02:09:06 - INFO - trainer -     f1 = 0.5154186100859396
Epoch: 100%|██████████| 30/30 [04:12<00:00,  8.43s/it]


NameError: name 'best_eval_result' is not defined

In [10]:
if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[args.hidden_layer_no] = best_acc_eval_result
    best_f1_eval_results_by_layer[args.hidden_layer_no] = best_f1_eval_result

07/23/2021 02:11:47 - INFO - __main__ -     acc = 0.7285714285714285
07/23/2021 02:11:47 - INFO - __main__ -     f1 = 0.5414583304317144


In [16]:
best_f1_eval_results_by_layer[11]

{'acc': 0.7285714285714285, 'f1': 0.5414583304317144}

### Output of Bert, output[11] Frozen 0

In [7]:
args.hidden_layer_no = 11
args.frozen_percent = 0
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_f1_eval_result

07/23/2021 19:45:43 - INFO - model -   Hidden Layer 11 frozen percentage 0
07/23/2021 19:45:48 - INFO - trainer -   ***** Running training *****
07/23/2021 19:45:48 - INFO - trainer -     Num examples = 3602
07/23/2021 19:45:48 - INFO - trainer -     Num Epochs = 30
07/23/2021 19:45:48 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 19:45:48 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 19:45:48 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 19:45:57 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 19:45:57 - INFO - trainer -     Num examples = 1120
07/23/2021 19:45:57 - INFO - trainer -     Batch size = 32
07/23/2021 19:45:58 - INFO - trainer -   ***** Eval results *****
07/23/2021 19:45:58 - INFO - trainer -     eval loss: 0.6670303591660091
07/23/2021 19:45:58 - INFO - trainer -     acc = 0.7446428571428572
07/23/2021 19:45:58 - INFO - trainer -     f1 = 0.569091

07/23/2021 19:48:06 - INFO - trainer -     acc = 0.8428571428571429
07/23/2021 19:48:06 - INFO - trainer -     f1 = 0.7614536764901411
07/23/2021 19:48:14 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 19:48:14 - INFO - trainer -     Num examples = 1120
07/23/2021 19:48:14 - INFO - trainer -     Batch size = 32
07/23/2021 19:48:15 - INFO - trainer -   ***** Eval results *****
07/23/2021 19:48:15 - INFO - trainer -     eval loss: 0.7576428955154759
07/23/2021 19:48:15 - INFO - trainer -     acc = 0.8419642857142857
07/23/2021 19:48:15 - INFO - trainer -     f1 = 0.7712913187104421
07/23/2021 19:48:22 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 19:48:22 - INFO - trainer -     Num examples = 1120
07/23/2021 19:48:22 - INFO - trainer -     Batch size = 32
07/23/2021 19:48:23 - INFO - trainer -   ***** Eval results *****
07/23/2021 19:48:23 - INFO - trainer -     eval loss: 0.8280671452571239
07/23/2021 19:48:23 - INFO - trainer -     acc = 0.8464285714285

07/23/2021 19:50:30 - INFO - trainer -     acc = 0.8392857142857143
07/23/2021 19:50:30 - INFO - trainer -     f1 = 0.771534570497844
Epoch:  50%|█████     | 15/30 [04:48<04:42, 18.86s/it]07/23/2021 19:50:37 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 19:50:37 - INFO - trainer -     Num examples = 1120
07/23/2021 19:50:37 - INFO - trainer -     Batch size = 32
07/23/2021 19:50:38 - INFO - trainer -   ***** Eval results *****
07/23/2021 19:50:39 - INFO - trainer -     eval loss: 1.0258610266526895
07/23/2021 19:50:39 - INFO - trainer -     acc = 0.8535714285714285
07/23/2021 19:50:39 - INFO - trainer -     f1 = 0.7844456286365387
07/23/2021 19:50:46 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 19:50:46 - INFO - trainer -     Num examples = 1120
07/23/2021 19:50:46 - INFO - trainer -     Batch size = 32
07/23/2021 19:50:47 - INFO - trainer -   ***** Eval results *****
07/23/2021 19:50:47 - INFO - trainer -     eval loss: 1.1033744797759157
07/23/2021 

07/23/2021 19:52:52 - INFO - trainer -     eval loss: 1.1764049646411356
07/23/2021 19:52:52 - INFO - trainer -     acc = 0.8419642857142857
07/23/2021 19:52:53 - INFO - trainer -     f1 = 0.7676595326778831
07/23/2021 19:53:00 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 19:53:00 - INFO - trainer -     Num examples = 1120
07/23/2021 19:53:00 - INFO - trainer -     Batch size = 32
07/23/2021 19:53:01 - INFO - trainer -   ***** Eval results *****
07/23/2021 19:53:01 - INFO - trainer -     eval loss: 1.2921959344829832
07/23/2021 19:53:01 - INFO - trainer -     acc = 0.8267857142857142
07/23/2021 19:53:01 - INFO - trainer -     f1 = 0.7467929817001023
Epoch:  77%|███████▋  | 23/30 [07:19<02:11, 18.72s/it]07/23/2021 19:53:08 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 19:53:08 - INFO - trainer -     Num examples = 1120
07/23/2021 19:53:08 - INFO - trainer -     Batch size = 32
07/23/2021 19:53:09 - INFO - trainer -   ***** Eval results *****
07/23/2021

07/23/2021 19:55:14 - INFO - trainer -   ***** Eval results *****
07/23/2021 19:55:14 - INFO - trainer -     eval loss: 1.2867140723192798
07/23/2021 19:55:14 - INFO - trainer -     acc = 0.83125
07/23/2021 19:55:14 - INFO - trainer -     f1 = 0.7477192801190586
Epoch: 100%|██████████| 30/30 [09:31<00:00, 19.06s/it]
07/23/2021 19:55:20 - INFO - __main__ -     acc = 0.8625
07/23/2021 19:55:20 - INFO - __main__ -     f1 = 0.7983648942079483


### Output of Bert, output[10] Frozen 0

In [10]:
args.hidden_layer_no = 10
args.frozen_percent = 0
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_f1_eval_result

07/23/2021 19:57:17 - INFO - model -   Hidden Layer 10 frozen percentage 0
07/23/2021 19:57:17 - INFO - trainer -   ***** Running training *****
07/23/2021 19:57:17 - INFO - trainer -     Num examples = 3602
07/23/2021 19:57:17 - INFO - trainer -     Num Epochs = 30
07/23/2021 19:57:17 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 19:57:17 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 19:57:17 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 19:57:24 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 19:57:24 - INFO - trainer -     Num examples = 1120
07/23/2021 19:57:24 - INFO - trainer -     Batch size = 32
07/23/2021 19:57:25 - INFO - trainer -   ***** Eval results *****
07/23/2021 19:57:25 - INFO - trainer -     eval loss: 0.6171426385641098
07/23/2021 19:57:25 - INFO - trainer -     acc = 0.7598214285714285
07/23/2021 19:57:25 - INFO - trainer -     f1 = 0.544099

07/23/2021 19:59:26 - INFO - trainer -     acc = 0.84375
07/23/2021 19:59:26 - INFO - trainer -     f1 = 0.7674824219218004
07/23/2021 19:59:33 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 19:59:33 - INFO - trainer -     Num examples = 1120
07/23/2021 19:59:33 - INFO - trainer -     Batch size = 32
07/23/2021 19:59:34 - INFO - trainer -   ***** Eval results *****
07/23/2021 19:59:34 - INFO - trainer -     eval loss: 0.9029278035940869
07/23/2021 19:59:34 - INFO - trainer -     acc = 0.8357142857142857
07/23/2021 19:59:34 - INFO - trainer -     f1 = 0.746209799056099
07/23/2021 19:59:41 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 19:59:41 - INFO - trainer -     Num examples = 1120
07/23/2021 19:59:41 - INFO - trainer -     Batch size = 32
07/23/2021 19:59:42 - INFO - trainer -   ***** Eval results *****
07/23/2021 19:59:42 - INFO - trainer -     eval loss: 0.8427069824188947
07/23/2021 19:59:42 - INFO - trainer -     acc = 0.8258928571428571
07/23/20

07/23/2021 20:01:42 - INFO - trainer -     eval loss: 1.0574512737744954
07/23/2021 20:01:42 - INFO - trainer -     acc = 0.8473214285714286
07/23/2021 20:01:42 - INFO - trainer -     f1 = 0.7618338693502092
Epoch:  50%|█████     | 15/30 [04:31<04:27, 17.86s/it]07/23/2021 20:01:49 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:01:49 - INFO - trainer -     Num examples = 1120
07/23/2021 20:01:49 - INFO - trainer -     Batch size = 32
07/23/2021 20:01:50 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:01:50 - INFO - trainer -     eval loss: 1.127197695094011
07/23/2021 20:01:50 - INFO - trainer -     acc = 0.8392857142857143
07/23/2021 20:01:50 - INFO - trainer -     f1 = 0.7488800030985567
07/23/2021 20:01:57 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:01:57 - INFO - trainer -     Num examples = 1120
07/23/2021 20:01:57 - INFO - trainer -     Batch size = 32
07/23/2021 20:01:58 - INFO - trainer -   ***** Eval results *****
07/23/2021 

07/23/2021 20:03:56 - INFO - trainer -     Num examples = 1120
07/23/2021 20:03:56 - INFO - trainer -     Batch size = 32
07/23/2021 20:03:58 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:03:58 - INFO - trainer -     eval loss: 1.1964952413125762
07/23/2021 20:03:58 - INFO - trainer -     acc = 0.8214285714285714
07/23/2021 20:03:58 - INFO - trainer -     f1 = 0.7437988925491799
07/23/2021 20:04:04 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:04:04 - INFO - trainer -     Num examples = 1120
07/23/2021 20:04:04 - INFO - trainer -     Batch size = 32
07/23/2021 20:04:06 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:04:06 - INFO - trainer -     eval loss: 1.1488033732889953
07/23/2021 20:04:06 - INFO - trainer -     acc = 0.8410714285714286
07/23/2021 20:04:06 - INFO - trainer -     f1 = 0.7631085945063519
Epoch:  77%|███████▋  | 23/30 [06:55<02:04, 17.83s/it]07/23/2021 20:04:12 - INFO - trainer -   ***** Running evaluation *****
07/23/2021

07/23/2021 20:06:12 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:06:12 - INFO - trainer -     Num examples = 1120
07/23/2021 20:06:12 - INFO - trainer -     Batch size = 32
07/23/2021 20:06:13 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:06:13 - INFO - trainer -     eval loss: 1.147089220837058
07/23/2021 20:06:13 - INFO - trainer -     acc = 0.8419642857142857
07/23/2021 20:06:13 - INFO - trainer -     f1 = 0.7634407069156728
Epoch: 100%|██████████| 30/30 [09:01<00:00, 18.04s/it]
07/23/2021 20:06:18 - INFO - __main__ -     acc = 0.8526785714285714
07/23/2021 20:06:18 - INFO - __main__ -     f1 = 0.7791258718917419


### Output of Bert, output[9] Frozen 0

In [11]:
args.hidden_layer_no = 9
args.frozen_percent = 0
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_f1_eval_result

07/23/2021 20:08:17 - INFO - model -   Hidden Layer 9 frozen percentage 0
07/23/2021 20:08:18 - INFO - trainer -   ***** Running training *****
07/23/2021 20:08:18 - INFO - trainer -     Num examples = 3602
07/23/2021 20:08:18 - INFO - trainer -     Num Epochs = 30
07/23/2021 20:08:18 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 20:08:18 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 20:08:18 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 20:08:24 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:08:24 - INFO - trainer -     Num examples = 1120
07/23/2021 20:08:24 - INFO - trainer -     Batch size = 32
07/23/2021 20:08:25 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:08:25 - INFO - trainer -     eval loss: 0.7272457633699689
07/23/2021 20:08:25 - INFO - trainer -     acc = 0.7125
07/23/2021 20:08:25 - INFO - trainer -     f1 = 0.47492765327411
07/2

07/23/2021 20:10:23 - INFO - trainer -     acc = 0.8366071428571429
07/23/2021 20:10:23 - INFO - trainer -     f1 = 0.7532209364928567
07/23/2021 20:10:29 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:10:29 - INFO - trainer -     Num examples = 1120
07/23/2021 20:10:29 - INFO - trainer -     Batch size = 32
07/23/2021 20:10:30 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:10:30 - INFO - trainer -     eval loss: 0.9014543068728277
07/23/2021 20:10:30 - INFO - trainer -     acc = 0.8303571428571429
07/23/2021 20:10:30 - INFO - trainer -     f1 = 0.7375290243049806
07/23/2021 20:10:37 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:10:37 - INFO - trainer -     Num examples = 1120
07/23/2021 20:10:37 - INFO - trainer -     Batch size = 32
07/23/2021 20:10:38 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:10:38 - INFO - trainer -     eval loss: 0.8527425945071238
07/23/2021 20:10:38 - INFO - trainer -     acc = 0.8419642857142

07/23/2021 20:12:33 - INFO - trainer -     eval loss: 1.1607451354082774
07/23/2021 20:12:33 - INFO - trainer -     acc = 0.8392857142857143
07/23/2021 20:12:33 - INFO - trainer -     f1 = 0.7568442103281919
Epoch:  50%|█████     | 15/30 [04:21<04:17, 17.17s/it]07/23/2021 20:12:39 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:12:39 - INFO - trainer -     Num examples = 1120
07/23/2021 20:12:39 - INFO - trainer -     Batch size = 32
07/23/2021 20:12:41 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:12:41 - INFO - trainer -     eval loss: 1.1901556309705483
07/23/2021 20:12:41 - INFO - trainer -     acc = 0.8401785714285714
07/23/2021 20:12:41 - INFO - trainer -     f1 = 0.7593674723120128
07/23/2021 20:12:47 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:12:47 - INFO - trainer -     Num examples = 1120
07/23/2021 20:12:47 - INFO - trainer -     Batch size = 32
07/23/2021 20:12:48 - INFO - trainer -   ***** Eval results *****
07/23/2021

07/23/2021 20:14:42 - INFO - trainer -     Batch size = 32
07/23/2021 20:14:43 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:14:43 - INFO - trainer -     eval loss: 1.2768271582144994
07/23/2021 20:14:43 - INFO - trainer -     acc = 0.825
07/23/2021 20:14:43 - INFO - trainer -     f1 = 0.7358721199815194
07/23/2021 20:14:50 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:14:50 - INFO - trainer -     Num examples = 1120
07/23/2021 20:14:50 - INFO - trainer -     Batch size = 32
07/23/2021 20:14:51 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:14:51 - INFO - trainer -     eval loss: 1.275411521029933
07/23/2021 20:14:51 - INFO - trainer -     acc = 0.8241071428571428
07/23/2021 20:14:51 - INFO - trainer -     f1 = 0.7350663732647691
Epoch:  77%|███████▋  | 23/30 [06:39<02:00, 17.14s/it]07/23/2021 20:14:57 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:14:57 - INFO - trainer -     Num examples = 1120
07/23/2021 20:14:57 - IN

07/23/2021 20:16:52 - INFO - trainer -     Num examples = 1120
07/23/2021 20:16:52 - INFO - trainer -     Batch size = 32
07/23/2021 20:16:53 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:16:53 - INFO - trainer -     eval loss: 1.3359377365027154
07/23/2021 20:16:53 - INFO - trainer -     acc = 0.8151785714285714
07/23/2021 20:16:53 - INFO - trainer -     f1 = 0.7465585783567249
Epoch: 100%|██████████| 30/30 [08:40<00:00, 17.35s/it]
07/23/2021 20:16:58 - INFO - __main__ -     acc = 0.8482142857142857
07/23/2021 20:16:58 - INFO - __main__ -     f1 = 0.7775968480864099


### Output of Bert, output[8] Frozen 0

In [14]:
args.hidden_layer_no = 8
args.frozen_percent = 0
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_f1_eval_result

07/23/2021 20:20:50 - INFO - model -   Hidden Layer 8 frozen percentage 0
07/23/2021 20:20:51 - INFO - trainer -   ***** Running training *****
07/23/2021 20:20:51 - INFO - trainer -     Num examples = 3602
07/23/2021 20:20:51 - INFO - trainer -     Num Epochs = 30
07/23/2021 20:20:51 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 20:20:51 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 20:20:51 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 20:20:57 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:20:57 - INFO - trainer -     Num examples = 1120
07/23/2021 20:20:57 - INFO - trainer -     Batch size = 32
07/23/2021 20:20:58 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:20:58 - INFO - trainer -     eval loss: 0.8833903431892395
07/23/2021 20:20:58 - INFO - trainer -     acc = 0.6553571428571429
07/23/2021 20:20:58 - INFO - trainer -     f1 = 0.3150021

07/23/2021 20:22:47 - INFO - trainer -     acc = 0.81875
07/23/2021 20:22:47 - INFO - trainer -     f1 = 0.7288510698557097
07/23/2021 20:22:53 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:22:53 - INFO - trainer -     Num examples = 1120
07/23/2021 20:22:53 - INFO - trainer -     Batch size = 32
07/23/2021 20:22:55 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:22:55 - INFO - trainer -     eval loss: 1.0201144171479557
07/23/2021 20:22:55 - INFO - trainer -     acc = 0.8142857142857143
07/23/2021 20:22:55 - INFO - trainer -     f1 = 0.7235627125006984
07/23/2021 20:23:01 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:23:01 - INFO - trainer -     Num examples = 1120
07/23/2021 20:23:01 - INFO - trainer -     Batch size = 32
07/23/2021 20:23:02 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:23:02 - INFO - trainer -     eval loss: 0.9443445054548127
07/23/2021 20:23:02 - INFO - trainer -     acc = 0.8044642857142857
07/23/2

07/23/2021 20:24:50 - INFO - trainer -     eval loss: 1.3706618103597845
07/23/2021 20:24:50 - INFO - trainer -     acc = 0.8089285714285714
07/23/2021 20:24:50 - INFO - trainer -     f1 = 0.701372544318671
Epoch:  50%|█████     | 15/30 [04:04<04:01, 16.07s/it]07/23/2021 20:24:56 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:24:56 - INFO - trainer -     Num examples = 1120
07/23/2021 20:24:56 - INFO - trainer -     Batch size = 32
07/23/2021 20:24:57 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:24:57 - INFO - trainer -     eval loss: 1.21262905725411
07/23/2021 20:24:57 - INFO - trainer -     acc = 0.8160714285714286
07/23/2021 20:24:57 - INFO - trainer -     f1 = 0.7218003236683895
07/23/2021 20:25:03 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:25:03 - INFO - trainer -     Num examples = 1120
07/23/2021 20:25:03 - INFO - trainer -     Batch size = 32
07/23/2021 20:25:04 - INFO - trainer -   ***** Eval results *****
07/23/2021 20

07/23/2021 20:26:51 - INFO - trainer -     Batch size = 32
07/23/2021 20:26:52 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:26:52 - INFO - trainer -     eval loss: 1.4046989509037562
07/23/2021 20:26:52 - INFO - trainer -     acc = 0.8214285714285714
07/23/2021 20:26:52 - INFO - trainer -     f1 = 0.7359142948640968
07/23/2021 20:26:58 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:26:58 - INFO - trainer -     Num examples = 1120
07/23/2021 20:26:58 - INFO - trainer -     Batch size = 32
07/23/2021 20:27:00 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:27:00 - INFO - trainer -     eval loss: 1.4264639983153238
07/23/2021 20:27:00 - INFO - trainer -     acc = 0.8223214285714285
07/23/2021 20:27:00 - INFO - trainer -     f1 = 0.7335601920703959
Epoch:  77%|███████▋  | 23/30 [06:14<01:52, 16.10s/it]07/23/2021 20:27:06 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:27:06 - INFO - trainer -     Num examples = 1120
07/23/2021

07/23/2021 20:28:53 - INFO - trainer -     Num examples = 1120
07/23/2021 20:28:53 - INFO - trainer -     Batch size = 32
07/23/2021 20:28:54 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:28:54 - INFO - trainer -     eval loss: 1.3370523265355068
07/23/2021 20:28:54 - INFO - trainer -     acc = 0.8232142857142857
07/23/2021 20:28:54 - INFO - trainer -     f1 = 0.73622635274016
Epoch: 100%|██████████| 30/30 [08:08<00:00, 16.28s/it]
07/23/2021 20:28:59 - INFO - __main__ -     acc = 0.8383928571428572
07/23/2021 20:28:59 - INFO - __main__ -     f1 = 0.7648445547046677


### Output of Bert, output[7] Frozen 0

In [18]:
args.hidden_layer_no = 7
args.frozen_percent = 0
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_f1_eval_result

07/23/2021 20:39:23 - INFO - model -   Hidden Layer 7 frozen percentage 0
07/23/2021 20:39:23 - INFO - trainer -   ***** Running training *****
07/23/2021 20:39:23 - INFO - trainer -     Num examples = 3602
07/23/2021 20:39:23 - INFO - trainer -     Num Epochs = 30
07/23/2021 20:39:23 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 20:39:23 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 20:39:23 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 20:39:29 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:39:29 - INFO - trainer -     Num examples = 1120
07/23/2021 20:39:29 - INFO - trainer -     Batch size = 32
07/23/2021 20:39:30 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:39:30 - INFO - trainer -     eval loss: 0.8479713116373334
07/23/2021 20:39:30 - INFO - trainer -     acc = 0.6660714285714285
07/23/2021 20:39:30 - INFO - trainer -     f1 = 0.3602930

07/23/2021 20:41:13 - INFO - trainer -     acc = 0.7982142857142858
07/23/2021 20:41:13 - INFO - trainer -     f1 = 0.7077369342335583
07/23/2021 20:41:19 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:41:19 - INFO - trainer -     Num examples = 1120
07/23/2021 20:41:19 - INFO - trainer -     Batch size = 32
07/23/2021 20:41:20 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:41:20 - INFO - trainer -     eval loss: 0.9574142373033933
07/23/2021 20:41:20 - INFO - trainer -     acc = 0.8
07/23/2021 20:41:20 - INFO - trainer -     f1 = 0.706666990387833
07/23/2021 20:41:26 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:41:26 - INFO - trainer -     Num examples = 1120
07/23/2021 20:41:26 - INFO - trainer -     Batch size = 32
07/23/2021 20:41:27 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:41:27 - INFO - trainer -     eval loss: 0.8444215386041574
07/23/2021 20:41:27 - INFO - trainer -     acc = 0.8053571428571429
07/23/2021 2

07/23/2021 20:43:09 - INFO - trainer -     eval loss: 1.3268335704824754
07/23/2021 20:43:09 - INFO - trainer -     acc = 0.7991071428571429
07/23/2021 20:43:09 - INFO - trainer -     f1 = 0.6961996468548409
Epoch:  50%|█████     | 15/30 [03:50<03:47, 15.17s/it]07/23/2021 20:43:15 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:43:15 - INFO - trainer -     Num examples = 1120
07/23/2021 20:43:15 - INFO - trainer -     Batch size = 32
07/23/2021 20:43:16 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:43:16 - INFO - trainer -     eval loss: 1.3430471451953054
07/23/2021 20:43:16 - INFO - trainer -     acc = 0.8151785714285714
07/23/2021 20:43:16 - INFO - trainer -     f1 = 0.7189178440843671
07/23/2021 20:43:21 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:43:21 - INFO - trainer -     Num examples = 1120
07/23/2021 20:43:21 - INFO - trainer -     Batch size = 32
07/23/2021 20:43:23 - INFO - trainer -   ***** Eval results *****
07/23/2021

07/23/2021 20:45:02 - INFO - trainer -     Batch size = 32
07/23/2021 20:45:04 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:45:04 - INFO - trainer -     eval loss: 1.5809124173746179
07/23/2021 20:45:04 - INFO - trainer -     acc = 0.8026785714285715
07/23/2021 20:45:04 - INFO - trainer -     f1 = 0.6846510818117539
07/23/2021 20:45:09 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:45:09 - INFO - trainer -     Num examples = 1120
07/23/2021 20:45:09 - INFO - trainer -     Batch size = 32
07/23/2021 20:45:10 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:45:10 - INFO - trainer -     eval loss: 1.5788362311008053
07/23/2021 20:45:10 - INFO - trainer -     acc = 0.8026785714285715
07/23/2021 20:45:10 - INFO - trainer -     f1 = 0.6867663063257413
Epoch:  77%|███████▋  | 23/30 [05:52<01:45, 15.09s/it]07/23/2021 20:45:16 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:45:16 - INFO - trainer -     Num examples = 1120
07/23/2021

07/23/2021 20:46:57 - INFO - trainer -     Num examples = 1120
07/23/2021 20:46:57 - INFO - trainer -     Batch size = 32
07/23/2021 20:46:58 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:46:58 - INFO - trainer -     eval loss: 1.6074872690818405
07/23/2021 20:46:58 - INFO - trainer -     acc = 0.80625
07/23/2021 20:46:58 - INFO - trainer -     f1 = 0.7083510035240099
Epoch: 100%|██████████| 30/30 [07:39<00:00, 15.31s/it]
07/23/2021 20:47:03 - INFO - __main__ -     acc = 0.8258928571428571
07/23/2021 20:47:03 - INFO - __main__ -     f1 = 0.7403418274383831


### Output of Bert, output[6] Frozen 0

In [20]:
args.hidden_layer_no = 6
args.frozen_percent = 0
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_f1_eval_result

07/23/2021 20:49:21 - INFO - model -   Hidden Layer 6 frozen percentage 0
07/23/2021 20:49:21 - INFO - trainer -   ***** Running training *****
07/23/2021 20:49:21 - INFO - trainer -     Num examples = 3602
07/23/2021 20:49:21 - INFO - trainer -     Num Epochs = 30
07/23/2021 20:49:21 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 20:49:21 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 20:49:21 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 20:49:26 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:49:26 - INFO - trainer -     Num examples = 1120
07/23/2021 20:49:26 - INFO - trainer -     Batch size = 32
07/23/2021 20:49:27 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:49:27 - INFO - trainer -     eval loss: 0.8704926252365113
07/23/2021 20:49:27 - INFO - trainer -     acc = 0.6553571428571429
07/23/2021 20:49:27 - INFO - trainer -     f1 = 0.2921050

07/23/2021 20:51:05 - INFO - trainer -     acc = 0.7910714285714285
07/23/2021 20:51:05 - INFO - trainer -     f1 = 0.6843914261807607
07/23/2021 20:51:10 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:51:10 - INFO - trainer -     Num examples = 1120
07/23/2021 20:51:10 - INFO - trainer -     Batch size = 32
07/23/2021 20:51:11 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:51:11 - INFO - trainer -     eval loss: 0.9534122432981219
07/23/2021 20:51:11 - INFO - trainer -     acc = 0.7794642857142857
07/23/2021 20:51:11 - INFO - trainer -     f1 = 0.6866759790027444
07/23/2021 20:51:17 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:51:17 - INFO - trainer -     Num examples = 1120
07/23/2021 20:51:17 - INFO - trainer -     Batch size = 32
07/23/2021 20:51:18 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:51:18 - INFO - trainer -     eval loss: 0.8575881902660643
07/23/2021 20:51:18 - INFO - trainer -     acc = 0.7883928571428

07/23/2021 20:52:54 - INFO - trainer -     eval loss: 1.4575537345239094
07/23/2021 20:52:54 - INFO - trainer -     acc = 0.7866071428571428
07/23/2021 20:52:54 - INFO - trainer -     f1 = 0.6938291755193164
Epoch:  50%|█████     | 15/30 [03:37<03:33, 14.26s/it]07/23/2021 20:52:59 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:52:59 - INFO - trainer -     Num examples = 1120
07/23/2021 20:52:59 - INFO - trainer -     Batch size = 32
07/23/2021 20:53:00 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:53:00 - INFO - trainer -     eval loss: 1.506428984138516
07/23/2021 20:53:00 - INFO - trainer -     acc = 0.7857142857142857
07/23/2021 20:53:00 - INFO - trainer -     f1 = 0.6765273130267606
07/23/2021 20:53:05 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:53:05 - INFO - trainer -     Num examples = 1120
07/23/2021 20:53:05 - INFO - trainer -     Batch size = 32
07/23/2021 20:53:06 - INFO - trainer -   ***** Eval results *****
07/23/2021 

07/23/2021 20:54:42 - INFO - trainer -     Batch size = 32
07/23/2021 20:54:43 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:54:43 - INFO - trainer -     eval loss: 1.7340395251688148
07/23/2021 20:54:43 - INFO - trainer -     acc = 0.7866071428571428
07/23/2021 20:54:43 - INFO - trainer -     f1 = 0.6638747258283337
07/23/2021 20:54:48 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:54:48 - INFO - trainer -     Num examples = 1120
07/23/2021 20:54:48 - INFO - trainer -     Batch size = 32
07/23/2021 20:54:49 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:54:49 - INFO - trainer -     eval loss: 1.5915360525250435
07/23/2021 20:54:49 - INFO - trainer -     acc = 0.7955357142857142
07/23/2021 20:54:49 - INFO - trainer -     f1 = 0.6985318888553168
Epoch:  77%|███████▋  | 23/30 [05:33<01:40, 14.34s/it]07/23/2021 20:54:54 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:54:54 - INFO - trainer -     Num examples = 1120
07/23/2021

07/23/2021 20:56:32 - INFO - trainer -     Num examples = 1120
07/23/2021 20:56:32 - INFO - trainer -     Batch size = 32
07/23/2021 20:56:33 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:56:33 - INFO - trainer -     eval loss: 1.9711945048400334
07/23/2021 20:56:33 - INFO - trainer -     acc = 0.7830357142857143
07/23/2021 20:56:33 - INFO - trainer -     f1 = 0.6547557586860464
Epoch: 100%|██████████| 30/30 [07:15<00:00, 14.53s/it]
07/23/2021 20:56:37 - INFO - __main__ -     acc = 0.8089285714285714
07/23/2021 20:56:37 - INFO - __main__ -     f1 = 0.7116461710857843


### Output of Bert, output[5] Frozen 0

In [23]:
args.hidden_layer_no = 5
args.frozen_percent = 0
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_f1_eval_result

07/23/2021 20:58:11 - INFO - model -   Hidden Layer 5 frozen percentage 0
07/23/2021 20:58:11 - INFO - trainer -   ***** Running training *****
07/23/2021 20:58:11 - INFO - trainer -     Num examples = 3602
07/23/2021 20:58:11 - INFO - trainer -     Num Epochs = 30
07/23/2021 20:58:11 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 20:58:11 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 20:58:11 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 20:58:16 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:58:16 - INFO - trainer -     Num examples = 1120
07/23/2021 20:58:16 - INFO - trainer -     Batch size = 32
07/23/2021 20:58:17 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:58:17 - INFO - trainer -     eval loss: 0.8661828875541687
07/23/2021 20:58:17 - INFO - trainer -     acc = 0.6508928571428572
07/23/2021 20:58:17 - INFO - trainer -     f1 = 0.2725655

07/23/2021 20:59:50 - INFO - trainer -     acc = 0.7741071428571429
07/23/2021 20:59:50 - INFO - trainer -     f1 = 0.6589658896407363
07/23/2021 20:59:55 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 20:59:55 - INFO - trainer -     Num examples = 1120
07/23/2021 20:59:55 - INFO - trainer -     Batch size = 32
07/23/2021 20:59:56 - INFO - trainer -   ***** Eval results *****
07/23/2021 20:59:56 - INFO - trainer -     eval loss: 0.9493770616395133
07/23/2021 20:59:56 - INFO - trainer -     acc = 0.7553571428571428
07/23/2021 20:59:56 - INFO - trainer -     f1 = 0.6577875587693099
07/23/2021 21:00:01 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:00:01 - INFO - trainer -     Num examples = 1120
07/23/2021 21:00:01 - INFO - trainer -     Batch size = 32
07/23/2021 21:00:02 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:00:02 - INFO - trainer -     eval loss: 0.9017819799482822
07/23/2021 21:00:02 - INFO - trainer -     acc = 0.7678571428571

07/23/2021 21:01:33 - INFO - trainer -     eval loss: 1.4827377183628934
07/23/2021 21:01:33 - INFO - trainer -     acc = 0.7821428571428571
07/23/2021 21:01:33 - INFO - trainer -     f1 = 0.6640637298124588
Epoch:  50%|█████     | 15/30 [03:26<03:23, 13.54s/it]07/23/2021 21:01:38 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:01:38 - INFO - trainer -     Num examples = 1120
07/23/2021 21:01:38 - INFO - trainer -     Batch size = 32
07/23/2021 21:01:39 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:01:39 - INFO - trainer -     eval loss: 1.600597586695637
07/23/2021 21:01:39 - INFO - trainer -     acc = 0.7803571428571429
07/23/2021 21:01:39 - INFO - trainer -     f1 = 0.6678333216553033
07/23/2021 21:01:44 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:01:44 - INFO - trainer -     Num examples = 1120
07/23/2021 21:01:44 - INFO - trainer -     Batch size = 32
07/23/2021 21:01:45 - INFO - trainer -   ***** Eval results *****
07/23/2021 

07/23/2021 21:03:15 - INFO - trainer -     Batch size = 32
07/23/2021 21:03:17 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:03:17 - INFO - trainer -     eval loss: 1.8823651184086754
07/23/2021 21:03:17 - INFO - trainer -     acc = 0.7669642857142858
07/23/2021 21:03:17 - INFO - trainer -     f1 = 0.6450620338169695
07/23/2021 21:03:21 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:03:21 - INFO - trainer -     Num examples = 1120
07/23/2021 21:03:21 - INFO - trainer -     Batch size = 32
07/23/2021 21:03:23 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:03:23 - INFO - trainer -     eval loss: 1.9659954393926975
07/23/2021 21:03:23 - INFO - trainer -     acc = 0.7616071428571428
07/23/2021 21:03:23 - INFO - trainer -     f1 = 0.6526665985983239
Epoch:  77%|███████▋  | 23/30 [05:16<01:35, 13.62s/it]07/23/2021 21:03:28 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:03:28 - INFO - trainer -     Num examples = 1120
07/23/2021

07/23/2021 21:04:58 - INFO - trainer -     Num examples = 1120
07/23/2021 21:04:58 - INFO - trainer -     Batch size = 32
07/23/2021 21:04:59 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:04:59 - INFO - trainer -     eval loss: 2.1218604662501352
07/23/2021 21:04:59 - INFO - trainer -     acc = 0.7660714285714286
07/23/2021 21:04:59 - INFO - trainer -     f1 = 0.6266677384431217
Epoch: 100%|██████████| 30/30 [06:52<00:00, 13.74s/it]
07/23/2021 21:05:03 - INFO - __main__ -     acc = 0.7857142857142857
07/23/2021 21:05:03 - INFO - __main__ -     f1 = 0.6710740356981434


### Output of Bert, output[4] Frozen 0

In [24]:
args.hidden_layer_no = 4
args.frozen_percent = 0
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_f1_eval_result

07/23/2021 21:06:15 - INFO - model -   Hidden Layer 4 frozen percentage 0
07/23/2021 21:06:15 - INFO - trainer -   ***** Running training *****
07/23/2021 21:06:15 - INFO - trainer -     Num examples = 3602
07/23/2021 21:06:15 - INFO - trainer -     Num Epochs = 30
07/23/2021 21:06:15 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 21:06:15 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 21:06:15 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 21:06:20 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:06:20 - INFO - trainer -     Num examples = 1120
07/23/2021 21:06:20 - INFO - trainer -     Batch size = 32
07/23/2021 21:06:21 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:06:21 - INFO - trainer -     eval loss: 0.8319082813603538
07/23/2021 21:06:21 - INFO - trainer -     acc = 0.6580357142857143
07/23/2021 21:06:21 - INFO - trainer -     f1 = 0.3089583

07/23/2021 21:07:46 - INFO - trainer -     acc = 0.7625
07/23/2021 21:07:46 - INFO - trainer -     f1 = 0.6357491249210995
07/23/2021 21:07:51 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:07:51 - INFO - trainer -     Num examples = 1120
07/23/2021 21:07:51 - INFO - trainer -     Batch size = 32
07/23/2021 21:07:52 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:07:52 - INFO - trainer -     eval loss: 0.9001162192651204
07/23/2021 21:07:52 - INFO - trainer -     acc = 0.7651785714285714
07/23/2021 21:07:52 - INFO - trainer -     f1 = 0.6654542183358543
07/23/2021 21:07:57 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:07:57 - INFO - trainer -     Num examples = 1120
07/23/2021 21:07:57 - INFO - trainer -     Batch size = 32
07/23/2021 21:07:58 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:07:58 - INFO - trainer -     eval loss: 0.7961768954992294
07/23/2021 21:07:58 - INFO - trainer -     acc = 0.7732142857142857
07/23/20

07/23/2021 21:09:24 - INFO - trainer -     eval loss: 1.5287791238299437
07/23/2021 21:09:24 - INFO - trainer -     acc = 0.7803571428571429
07/23/2021 21:09:24 - INFO - trainer -     f1 = 0.6739658147843796
Epoch:  50%|█████     | 15/30 [03:12<03:09, 12.61s/it]07/23/2021 21:09:29 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:09:29 - INFO - trainer -     Num examples = 1120
07/23/2021 21:09:29 - INFO - trainer -     Batch size = 32
07/23/2021 21:09:30 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:09:30 - INFO - trainer -     eval loss: 1.5118274636034454
07/23/2021 21:09:30 - INFO - trainer -     acc = 0.7758928571428572
07/23/2021 21:09:30 - INFO - trainer -     f1 = 0.6715384175010634
07/23/2021 21:09:34 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:09:34 - INFO - trainer -     Num examples = 1120
07/23/2021 21:09:34 - INFO - trainer -     Batch size = 32
07/23/2021 21:09:35 - INFO - trainer -   ***** Eval results *****
07/23/2021

07/23/2021 21:10:58 - INFO - trainer -     Batch size = 32
07/23/2021 21:10:59 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:10:59 - INFO - trainer -     eval loss: 1.9058528916112014
07/23/2021 21:10:59 - INFO - trainer -     acc = 0.7535714285714286
07/23/2021 21:10:59 - INFO - trainer -     f1 = 0.6429390255885766
07/23/2021 21:11:03 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:11:03 - INFO - trainer -     Num examples = 1120
07/23/2021 21:11:03 - INFO - trainer -     Batch size = 32
07/23/2021 21:11:05 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:11:05 - INFO - trainer -     eval loss: 1.868526771238872
07/23/2021 21:11:05 - INFO - trainer -     acc = 0.7794642857142857
07/23/2021 21:11:05 - INFO - trainer -     f1 = 0.675703630251255
Epoch:  77%|███████▋  | 23/30 [04:53<01:27, 12.46s/it]07/23/2021 21:11:09 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:11:09 - INFO - trainer -     Num examples = 1120
07/23/2021 2

07/23/2021 21:12:33 - INFO - trainer -     Num examples = 1120
07/23/2021 21:12:33 - INFO - trainer -     Batch size = 32
07/23/2021 21:12:34 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:12:34 - INFO - trainer -     eval loss: 2.4177176679883683
07/23/2021 21:12:34 - INFO - trainer -     acc = 0.7491071428571429
07/23/2021 21:12:34 - INFO - trainer -     f1 = 0.5897706107003108
Epoch: 100%|██████████| 30/30 [06:21<00:00, 12.73s/it]
07/23/2021 21:12:37 - INFO - __main__ -     acc = 0.7848214285714286
07/23/2021 21:12:37 - INFO - __main__ -     f1 = 0.6839390708793694


### Output of Bert, output[3] Frozen 0

In [25]:
args.hidden_layer_no = 3
args.frozen_percent = 0
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_f1_eval_result

07/23/2021 21:14:20 - INFO - model -   Hidden Layer 3 frozen percentage 0
07/23/2021 21:14:20 - INFO - trainer -   ***** Running training *****
07/23/2021 21:14:20 - INFO - trainer -     Num examples = 3602
07/23/2021 21:14:20 - INFO - trainer -     Num Epochs = 30
07/23/2021 21:14:20 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 21:14:20 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 21:14:20 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 21:14:24 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:14:24 - INFO - trainer -     Num examples = 1120
07/23/2021 21:14:24 - INFO - trainer -     Batch size = 32
07/23/2021 21:14:25 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:14:25 - INFO - trainer -     eval loss: 0.8657854846545628
07/23/2021 21:14:25 - INFO - trainer -     acc = 0.65
07/23/2021 21:14:25 - INFO - trainer -     f1 = 0.26262626262626265
07/

07/23/2021 21:15:44 - INFO - trainer -     acc = 0.7732142857142857
07/23/2021 21:15:44 - INFO - trainer -     f1 = 0.6541207509951704
07/23/2021 21:15:48 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:15:48 - INFO - trainer -     Num examples = 1120
07/23/2021 21:15:48 - INFO - trainer -     Batch size = 32
07/23/2021 21:15:49 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:15:49 - INFO - trainer -     eval loss: 0.8196573532053403
07/23/2021 21:15:49 - INFO - trainer -     acc = 0.7508928571428571
07/23/2021 21:15:49 - INFO - trainer -     f1 = 0.6553107338961442
07/23/2021 21:15:53 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:15:53 - INFO - trainer -     Num examples = 1120
07/23/2021 21:15:53 - INFO - trainer -     Batch size = 32
07/23/2021 21:15:54 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:15:54 - INFO - trainer -     eval loss: 0.752895949780941
07/23/2021 21:15:54 - INFO - trainer -     acc = 0.76071428571428

07/23/2021 21:17:13 - INFO - trainer -     eval loss: 1.2978238808257239
07/23/2021 21:17:13 - INFO - trainer -     acc = 0.7535714285714286
07/23/2021 21:17:13 - INFO - trainer -     f1 = 0.652335819285672
Epoch:  50%|█████     | 15/30 [02:56<02:54, 11.64s/it]07/23/2021 21:17:17 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:17:17 - INFO - trainer -     Num examples = 1120
07/23/2021 21:17:17 - INFO - trainer -     Batch size = 32
07/23/2021 21:17:18 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:17:18 - INFO - trainer -     eval loss: 1.4562470740505626
07/23/2021 21:17:18 - INFO - trainer -     acc = 0.7642857142857142
07/23/2021 21:17:18 - INFO - trainer -     f1 = 0.6407805328036542
07/23/2021 21:17:22 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:17:22 - INFO - trainer -     Num examples = 1120
07/23/2021 21:17:22 - INFO - trainer -     Batch size = 32
07/23/2021 21:17:23 - INFO - trainer -   ***** Eval results *****
07/23/2021 

07/23/2021 21:18:40 - INFO - trainer -     Batch size = 32
07/23/2021 21:18:41 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:18:41 - INFO - trainer -     eval loss: 1.8135411396622658
07/23/2021 21:18:41 - INFO - trainer -     acc = 0.7508928571428571
07/23/2021 21:18:41 - INFO - trainer -     f1 = 0.6200951818854632
07/23/2021 21:18:45 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:18:45 - INFO - trainer -     Num examples = 1120
07/23/2021 21:18:45 - INFO - trainer -     Batch size = 32
07/23/2021 21:18:46 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:18:46 - INFO - trainer -     eval loss: 1.8425300160689013
07/23/2021 21:18:46 - INFO - trainer -     acc = 0.7616071428571428
07/23/2021 21:18:46 - INFO - trainer -     f1 = 0.6370177578142552
Epoch:  77%|███████▋  | 23/30 [04:29<01:21, 11.57s/it]07/23/2021 21:18:50 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:18:50 - INFO - trainer -     Num examples = 1120
07/23/2021

07/23/2021 21:20:09 - INFO - trainer -     Num examples = 1120
07/23/2021 21:20:09 - INFO - trainer -     Batch size = 32
07/23/2021 21:20:10 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:20:10 - INFO - trainer -     eval loss: 2.1481233924627303
07/23/2021 21:20:10 - INFO - trainer -     acc = 0.7598214285714285
07/23/2021 21:20:10 - INFO - trainer -     f1 = 0.640020203729202
Epoch: 100%|██████████| 30/30 [05:53<00:00, 11.77s/it]
07/23/2021 21:20:13 - INFO - __main__ -     acc = 0.7848214285714286
07/23/2021 21:20:13 - INFO - __main__ -     f1 = 0.6624438214965042


### Output of Bert, output[2] Frozen 0

In [26]:
args.hidden_layer_no = 2
args.frozen_percent = 0
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_f1_eval_result

07/23/2021 21:22:12 - INFO - model -   Hidden Layer 2 frozen percentage 0
07/23/2021 21:22:12 - INFO - trainer -   ***** Running training *****
07/23/2021 21:22:12 - INFO - trainer -     Num examples = 3602
07/23/2021 21:22:12 - INFO - trainer -     Num Epochs = 30
07/23/2021 21:22:12 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 21:22:12 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 21:22:12 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 21:22:16 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:22:16 - INFO - trainer -     Num examples = 1120
07/23/2021 21:22:16 - INFO - trainer -     Batch size = 32
07/23/2021 21:22:17 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:22:17 - INFO - trainer -     eval loss: 0.8925731267247882
07/23/2021 21:22:17 - INFO - trainer -     acc = 0.65
07/23/2021 21:22:17 - INFO - trainer -     f1 = 0.26262626262626265
07/

07/23/2021 21:23:30 - INFO - trainer -     acc = 0.76875
07/23/2021 21:23:30 - INFO - trainer -     f1 = 0.6592215434452567
07/23/2021 21:23:33 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:23:33 - INFO - trainer -     Num examples = 1120
07/23/2021 21:23:33 - INFO - trainer -     Batch size = 32
07/23/2021 21:23:34 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:23:34 - INFO - trainer -     eval loss: 0.7929618362869535
07/23/2021 21:23:34 - INFO - trainer -     acc = 0.7419642857142857
07/23/2021 21:23:34 - INFO - trainer -     f1 = 0.6551175683762319
07/23/2021 21:23:38 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:23:38 - INFO - trainer -     Num examples = 1120
07/23/2021 21:23:38 - INFO - trainer -     Batch size = 32
07/23/2021 21:23:39 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:23:39 - INFO - trainer -     eval loss: 0.7167954217110362
07/23/2021 21:23:39 - INFO - trainer -     acc = 0.7598214285714285
07/23/2

07/23/2021 21:24:51 - INFO - trainer -     acc = 0.7464285714285714
07/23/2021 21:24:51 - INFO - trainer -     f1 = 0.6368776405238977
Epoch:  50%|█████     | 15/30 [02:42<02:40, 10.73s/it]07/23/2021 21:24:55 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:24:55 - INFO - trainer -     Num examples = 1120
07/23/2021 21:24:55 - INFO - trainer -     Batch size = 32
07/23/2021 21:24:56 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:24:56 - INFO - trainer -     eval loss: 1.1326560307826314
07/23/2021 21:24:56 - INFO - trainer -     acc = 0.7392857142857143
07/23/2021 21:24:56 - INFO - trainer -     f1 = 0.6218349438744709
07/23/2021 21:25:00 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:25:00 - INFO - trainer -     Num examples = 1120
07/23/2021 21:25:00 - INFO - trainer -     Batch size = 32
07/23/2021 21:25:01 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:25:01 - INFO - trainer -     eval loss: 1.2421877913177013
07/23/2021

07/23/2021 21:26:14 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:26:14 - INFO - trainer -     eval loss: 1.744172774042402
07/23/2021 21:26:14 - INFO - trainer -     acc = 0.7303571428571428
07/23/2021 21:26:14 - INFO - trainer -     f1 = 0.601640215825361
07/23/2021 21:26:18 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:26:18 - INFO - trainer -     Num examples = 1120
07/23/2021 21:26:18 - INFO - trainer -     Batch size = 32
07/23/2021 21:26:19 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:26:19 - INFO - trainer -     eval loss: 1.951490506529808
07/23/2021 21:26:19 - INFO - trainer -     acc = 0.7348214285714286
07/23/2021 21:26:19 - INFO - trainer -     f1 = 0.6190599312516891
Epoch:  77%|███████▋  | 23/30 [04:10<01:16, 10.93s/it]07/23/2021 21:26:23 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:26:23 - INFO - trainer -     Num examples = 1120
07/23/2021 21:26:23 - INFO - trainer -     Batch size = 32
07/23/2021 21

07/23/2021 21:27:37 - INFO - trainer -     Batch size = 32
07/23/2021 21:27:38 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:27:38 - INFO - trainer -     eval loss: 2.2927623949944973
07/23/2021 21:27:38 - INFO - trainer -     acc = 0.7392857142857143
07/23/2021 21:27:38 - INFO - trainer -     f1 = 0.606029207680053
Epoch: 100%|██████████| 30/30 [05:28<00:00, 10.95s/it]
07/23/2021 21:27:41 - INFO - __main__ -     acc = 0.7758928571428572
07/23/2021 21:27:41 - INFO - __main__ -     f1 = 0.6818424550257919


### Output of Bert, output[1] Frozen 0

In [27]:
args.hidden_layer_no = 1
args.frozen_percent = 0
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_f1_eval_result

07/23/2021 21:29:08 - INFO - model -   Hidden Layer 1 frozen percentage 0
07/23/2021 21:29:08 - INFO - trainer -   ***** Running training *****
07/23/2021 21:29:08 - INFO - trainer -     Num examples = 3602
07/23/2021 21:29:08 - INFO - trainer -     Num Epochs = 30
07/23/2021 21:29:08 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 21:29:08 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 21:29:08 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 21:29:11 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:29:11 - INFO - trainer -     Num examples = 1120
07/23/2021 21:29:11 - INFO - trainer -     Batch size = 32
07/23/2021 21:29:12 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:29:12 - INFO - trainer -     eval loss: 0.9156643169266837
07/23/2021 21:29:12 - INFO - trainer -     acc = 0.65
07/23/2021 21:29:12 - INFO - trainer -     f1 = 0.26262626262626265
07/

07/23/2021 21:30:19 - INFO - trainer -     acc = 0.7535714285714286
07/23/2021 21:30:19 - INFO - trainer -     f1 = 0.6108851774543683
07/23/2021 21:30:23 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:30:23 - INFO - trainer -     Num examples = 1120
07/23/2021 21:30:23 - INFO - trainer -     Batch size = 32
07/23/2021 21:30:24 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:30:24 - INFO - trainer -     eval loss: 0.8492424811635698
07/23/2021 21:30:24 - INFO - trainer -     acc = 0.7321428571428571
07/23/2021 21:30:24 - INFO - trainer -     f1 = 0.6225384797293927
07/23/2021 21:30:27 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:30:27 - INFO - trainer -     Num examples = 1120
07/23/2021 21:30:27 - INFO - trainer -     Batch size = 32
07/23/2021 21:30:28 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:30:28 - INFO - trainer -     eval loss: 0.8078548388821738
07/23/2021 21:30:28 - INFO - trainer -     acc = 0.7491071428571

07/23/2021 21:31:35 - INFO - trainer -     eval loss: 1.200575001431363
07/23/2021 21:31:35 - INFO - trainer -     acc = 0.7303571428571428
07/23/2021 21:31:35 - INFO - trainer -     f1 = 0.5844185250359681
Epoch:  50%|█████     | 15/30 [02:30<02:29,  9.98s/it]07/23/2021 21:31:38 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:31:38 - INFO - trainer -     Num examples = 1120
07/23/2021 21:31:38 - INFO - trainer -     Batch size = 32
07/23/2021 21:31:40 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:31:40 - INFO - trainer -     eval loss: 1.265752311689513
07/23/2021 21:31:40 - INFO - trainer -     acc = 0.7383928571428572
07/23/2021 21:31:40 - INFO - trainer -     f1 = 0.6204036849308273
07/23/2021 21:31:43 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:31:43 - INFO - trainer -     Num examples = 1120
07/23/2021 21:31:43 - INFO - trainer -     Batch size = 32
07/23/2021 21:31:44 - INFO - trainer -   ***** Eval results *****
07/23/2021 2

07/23/2021 21:32:51 - INFO - trainer -     Batch size = 32
07/23/2021 21:32:52 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:32:52 - INFO - trainer -     eval loss: 1.7786988007170814
07/23/2021 21:32:52 - INFO - trainer -     acc = 0.7232142857142857
07/23/2021 21:32:52 - INFO - trainer -     f1 = 0.6004118420078314
07/23/2021 21:32:55 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:32:55 - INFO - trainer -     Num examples = 1120
07/23/2021 21:32:55 - INFO - trainer -     Batch size = 32
07/23/2021 21:32:57 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:32:57 - INFO - trainer -     eval loss: 1.8755180808050291
07/23/2021 21:32:57 - INFO - trainer -     acc = 0.7196428571428571
07/23/2021 21:32:57 - INFO - trainer -     f1 = 0.5802008200932794
Epoch:  77%|███████▋  | 23/30 [03:51<01:10, 10.07s/it]07/23/2021 21:33:00 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:33:00 - INFO - trainer -     Num examples = 1120
07/23/2021

07/23/2021 21:34:07 - INFO - trainer -     Num examples = 1120
07/23/2021 21:34:07 - INFO - trainer -     Batch size = 32
07/23/2021 21:34:08 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:34:08 - INFO - trainer -     eval loss: 2.409923778261457
07/23/2021 21:34:08 - INFO - trainer -     acc = 0.7071428571428572
07/23/2021 21:34:08 - INFO - trainer -     f1 = 0.5813107590456846
Epoch: 100%|██████████| 30/30 [05:02<00:00, 10.09s/it]
07/23/2021 21:34:10 - INFO - __main__ -     acc = 0.7633928571428571
07/23/2021 21:34:10 - INFO - __main__ -     f1 = 0.6241070849483733


### Output of Bert, output[0] Frozen 0

In [28]:
args.hidden_layer_no = 0
args.frozen_percent = 0
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))
        
    best_acc_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_acc_eval_result
    best_f1_eval_results_by_layer[(args.hidden_layer_no, args.frozen_percent)] = best_f1_eval_result

07/23/2021 21:35:12 - INFO - model -   Hidden Layer 0 frozen percentage 0
07/23/2021 21:35:12 - INFO - trainer -   ***** Running training *****
07/23/2021 21:35:12 - INFO - trainer -     Num examples = 3602
07/23/2021 21:35:12 - INFO - trainer -     Num Epochs = 30
07/23/2021 21:35:12 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 21:35:12 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 21:35:12 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 21:35:15 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:35:15 - INFO - trainer -     Num examples = 1120
07/23/2021 21:35:15 - INFO - trainer -     Batch size = 32
07/23/2021 21:35:16 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:35:16 - INFO - trainer -     eval loss: 0.8963853376252311
07/23/2021 21:35:16 - INFO - trainer -     acc = 0.65
07/23/2021 21:35:16 - INFO - trainer -     f1 = 0.26262626262626265
07/

07/23/2021 21:36:20 - INFO - trainer -     Num examples = 1120
07/23/2021 21:36:20 - INFO - trainer -     Batch size = 32
07/23/2021 21:36:21 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:36:21 - INFO - trainer -     eval loss: 0.896083174433027
07/23/2021 21:36:21 - INFO - trainer -     acc = 0.65
07/23/2021 21:36:21 - INFO - trainer -     f1 = 0.26262626262626265
07/23/2021 21:36:24 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:36:24 - INFO - trainer -     Num examples = 1120
07/23/2021 21:36:24 - INFO - trainer -     Batch size = 32
07/23/2021 21:36:25 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:36:25 - INFO - trainer -     eval loss: 0.9039764864104135
07/23/2021 21:36:25 - INFO - trainer -     acc = 0.65
07/23/2021 21:36:25 - INFO - trainer -     f1 = 0.26262626262626265
Epoch:  27%|██▋       | 8/30 [01:13<03:23,  9.24s/it]07/23/2021 21:36:28 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:36:28 - INFO - trainer -

07/23/2021 21:37:29 - INFO - trainer -     Batch size = 32
07/23/2021 21:37:30 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:37:30 - INFO - trainer -     eval loss: 0.8988484944616045
07/23/2021 21:37:30 - INFO - trainer -     acc = 0.65
07/23/2021 21:37:30 - INFO - trainer -     f1 = 0.26262626262626265
07/23/2021 21:37:33 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:37:33 - INFO - trainer -     Num examples = 1120
07/23/2021 21:37:33 - INFO - trainer -     Batch size = 32
07/23/2021 21:37:34 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:37:34 - INFO - trainer -     eval loss: 0.8950382990496499
07/23/2021 21:37:34 - INFO - trainer -     acc = 0.65
07/23/2021 21:37:34 - INFO - trainer -     f1 = 0.26262626262626265
07/23/2021 21:37:37 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:37:37 - INFO - trainer -     Num examples = 1120
07/23/2021 21:37:37 - INFO - trainer -     Batch size = 32
07/23/2021 21:37:38 - INFO - tr

07/23/2021 21:38:40 - INFO - trainer -     eval loss: 0.8954925809587752
07/23/2021 21:38:40 - INFO - trainer -     acc = 0.65
07/23/2021 21:38:40 - INFO - trainer -     f1 = 0.26262626262626265
Epoch:  77%|███████▋  | 23/30 [03:30<01:03,  9.12s/it]07/23/2021 21:38:43 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:38:43 - INFO - trainer -     Num examples = 1120
07/23/2021 21:38:43 - INFO - trainer -     Batch size = 32
07/23/2021 21:38:44 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:38:44 - INFO - trainer -     eval loss: 0.8979322229112897
07/23/2021 21:38:44 - INFO - trainer -     acc = 0.65
07/23/2021 21:38:44 - INFO - trainer -     f1 = 0.26262626262626265
07/23/2021 21:38:47 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 21:38:47 - INFO - trainer -     Num examples = 1120
07/23/2021 21:38:47 - INFO - trainer -     Batch size = 32
07/23/2021 21:38:48 - INFO - trainer -   ***** Eval results *****
07/23/2021 21:38:48 - INFO - trainer

## Two Layers

##  Layer 11 + Layer 12 Frozen 2 

In [3]:
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                        datefmt='%m/%d/%Y %H:%M:%S',
                        level=logging.INFO)
    
# Parse args

args_str = "--embedding_type bert --output_dir data/output-gcn --dropout 0.3 --hidden_size 200 --learning_rate 5e-5 --bert_model_dir ./test/saved_model --pure_bert --frozen_percent 0"
#args = parse_args(['--gat_our', '--highway', '--num_heads', '7', '--dropout', '0.8', '--output_dir',
#                   'output/r-gat', '--glove_dir', 'glove', '--cuda_id', '0'])
args = parse_args(args_str.split(' '))
check_args(args)

# Setup CUDA, GPU training
os.environ["CUDA_VISIBLE_DEVICES"] = args.cuda_id
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
args.device = device
logger.info('Device is %s', args.device)

# Set seed
set_seed(args)

# Bert, load pretrained model and tokenizer, check if neccesary to put bert here
if args.embedding_type == 'bert':
    tokenizer = BertTokenizer.from_pretrained(args.bert_model_dir)
    args.tokenizer = tokenizer

# Load datasets and vocabs
train_dataset, test_dataset, word_vocab, dep_tag_vocab, pos_tag_vocab= load_datasets_and_vocabs(args)



07/23/2021 03:49:46 - INFO - __main__ -   {'dataset_name': 'rest', 'output_dir': 'data/output-gcn', 'num_classes': 3, 'cuda_id': '3', 'seed': 2019, 'glove_dir': '/data1/SHENWZH/wordvec', 'bert_model_dir': './test/saved_model', 'pure_bert': True, 'pure_bert_layer_agg': False, 'pure_bert_layer_agg_list': '12', 'pure_bert_linear_layer_count': 2, 'gat_bert': False, 'highway': False, 'num_layers': 2, 'hidden_layer_no': 12, 'frozen_percent': 0.0, 'add_non_connect': True, 'multi_hop': True, 'max_hop': 4, 'num_heads': 6, 'dropout': 0.3, 'num_gcn_layers': 1, 'gcn_mem_dim': 300, 'gcn_dropout': 0.2, 'gat': False, 'gat_our': False, 'gat_attention_type': 'dotprod', 'embedding_type': 'bert', 'embedding_dim': 300, 'dep_relation_embed_dim': 300, 'hidden_size': 200, 'final_hidden_size': 300, 'num_mlps': 2, 'per_gpu_train_batch_size': 16, 'per_gpu_eval_batch_size': 32, 'gradient_accumulation_steps': 2, 'learning_rate': 5e-05, 'weight_decay': 0.0, 'adam_epsilon': 1e-08, 'max_grad_norm': 1.0, 'num_train_e

In [4]:
# Build Model
# model = Aspect_Text_Multi_Syntax_Encoding(args, dep_tag_vocab['len'], pos_tag_vocab['len'])
if args.pure_bert:
    model = Pure_Bert(args)
elif args.gat_bert:
    model = Aspect_Bert_GAT(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # R-GAT + Bert
elif args.gat_our:
    model = Aspect_Text_GAT_ours(args, dep_tag_vocab['len'], pos_tag_vocab['len']) # R-GAT with reshaped tree
else:
    model = Aspect_Text_GAT_only(args, dep_tag_vocab['len'], pos_tag_vocab['len'])  # original GAT with reshaped tree

model.to(args.device)
# Train


_, _,  all_eval_results = train(args, train_dataset, model, test_dataset)

if len(all_eval_results):
    best_acc_eval_result = max(all_eval_results, key=lambda x: x['acc']) 
    best_f1_eval_result = max(all_eval_results, key=lambda x: x['f1'])
    
    for key in sorted(best_acc_eval_result.keys()):
        logger.info("  %s = %s", key, str(best_acc_eval_result[key]))


07/23/2021 03:52:08 - INFO - model -   Hidden Layer 11 & 12, FC1
07/23/2021 03:52:10 - INFO - trainer -   ***** Running training *****
07/23/2021 03:52:10 - INFO - trainer -     Num examples = 3602
07/23/2021 03:52:10 - INFO - trainer -     Num Epochs = 30
07/23/2021 03:52:10 - INFO - trainer -     Instantaneous batch size per GPU = 16
07/23/2021 03:52:10 - INFO - trainer -     Gradient Accumulation steps = 2
07/23/2021 03:52:10 - INFO - trainer -     Total optimization steps = 3390
Epoch:   0%|          | 0/30 [00:00<?, ?it/s]07/23/2021 03:52:18 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:52:18 - INFO - trainer -     Num examples = 1120
07/23/2021 03:52:18 - INFO - trainer -     Batch size = 32
07/23/2021 03:52:19 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:52:19 - INFO - trainer -     eval loss: 0.5427059139524187
07/23/2021 03:52:19 - INFO - trainer -     acc = 0.7910714285714285
07/23/2021 03:52:19 - INFO - trainer -     f1 = 0.6710150238294069

07/23/2021 03:54:31 - INFO - trainer -     acc = 0.8482142857142857
07/23/2021 03:54:31 - INFO - trainer -     f1 = 0.7717813872675819
07/23/2021 03:54:39 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:54:39 - INFO - trainer -     Num examples = 1120
07/23/2021 03:54:39 - INFO - trainer -     Batch size = 32
07/23/2021 03:54:40 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:54:40 - INFO - trainer -     eval loss: 0.7895552602214073
07/23/2021 03:54:40 - INFO - trainer -     acc = 0.85625
07/23/2021 03:54:40 - INFO - trainer -     f1 = 0.7878743668392203
07/23/2021 03:54:47 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:54:47 - INFO - trainer -     Num examples = 1120
07/23/2021 03:54:47 - INFO - trainer -     Batch size = 32
07/23/2021 03:54:49 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:54:49 - INFO - trainer -     eval loss: 0.8248617659422702
07/23/2021 03:54:49 - INFO - trainer -     acc = 0.8419642857142857
07/23/2

07/23/2021 03:56:58 - INFO - trainer -     eval loss: 1.1981529227736505
07/23/2021 03:56:58 - INFO - trainer -     acc = 0.8464285714285714
07/23/2021 03:56:58 - INFO - trainer -     f1 = 0.769884345530729
Epoch:  50%|█████     | 15/30 [04:54<04:49, 19.33s/it]07/23/2021 03:57:05 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:57:05 - INFO - trainer -     Num examples = 1120
07/23/2021 03:57:05 - INFO - trainer -     Batch size = 32
07/23/2021 03:57:07 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:57:07 - INFO - trainer -     eval loss: 1.0475693627543348
07/23/2021 03:57:07 - INFO - trainer -     acc = 0.8598214285714286
07/23/2021 03:57:07 - INFO - trainer -     f1 = 0.7957430951537852
07/23/2021 03:57:14 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:57:14 - INFO - trainer -     Num examples = 1120
07/23/2021 03:57:14 - INFO - trainer -     Batch size = 32
07/23/2021 03:57:15 - INFO - trainer -   ***** Eval results *****
07/23/2021 

07/23/2021 03:59:23 - INFO - trainer -     Batch size = 32
07/23/2021 03:59:24 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:59:24 - INFO - trainer -     eval loss: 1.0949547391002332
07/23/2021 03:59:24 - INFO - trainer -     acc = 0.8473214285714286
07/23/2021 03:59:24 - INFO - trainer -     f1 = 0.7777524586809662
07/23/2021 03:59:32 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:59:32 - INFO - trainer -     Num examples = 1120
07/23/2021 03:59:32 - INFO - trainer -     Batch size = 32
07/23/2021 03:59:33 - INFO - trainer -   ***** Eval results *****
07/23/2021 03:59:33 - INFO - trainer -     eval loss: 1.1553801152589065
07/23/2021 03:59:33 - INFO - trainer -     acc = 0.8482142857142857
07/23/2021 03:59:33 - INFO - trainer -     f1 = 0.7746263074148941
Epoch:  77%|███████▋  | 23/30 [07:29<02:15, 19.29s/it]07/23/2021 03:59:40 - INFO - trainer -   ***** Running evaluation *****
07/23/2021 03:59:40 - INFO - trainer -     Num examples = 1120
07/23/2021

07/23/2021 04:01:49 - INFO - trainer -     Num examples = 1120
07/23/2021 04:01:49 - INFO - trainer -     Batch size = 32
07/23/2021 04:01:50 - INFO - trainer -   ***** Eval results *****
07/23/2021 04:01:50 - INFO - trainer -     eval loss: 1.540078211121155
07/23/2021 04:01:50 - INFO - trainer -     acc = 0.8321428571428572
07/23/2021 04:01:50 - INFO - trainer -     f1 = 0.726141641118113
Epoch: 100%|██████████| 30/30 [09:45<00:00, 19.53s/it]
07/23/2021 04:01:56 - INFO - __main__ -     acc = 0.8598214285714286
07/23/2021 04:01:56 - INFO - __main__ -     f1 = 0.7957430951537852
